In [ ]:
!python --version

Python 3.10.11


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [25]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install --upgrade cupy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 385, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 385 (delta 86), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (385/385), 105.74 KiB | 7.55 MiB/s, done.
Resolving deltas: 100% (188/188), done.


In [ ]:
!python rapidsai-csp-utils/colab/pip-install.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.9 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.4/428.4 kB 46.6 MB/s eta 0:00:00


In [ ]:
# install pybaseball for data
! pip install pybaseball

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.0/420.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.5/312.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 50.2 MB/s eta 0:00:00


In [1]:
# import pybaseball
# from pybaseball import schedule_and_record
import pandas as pd
import cupy as cp
import numpy as np
# import cudf
import glob
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# team batting advanced stats per year
files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/Batting/per season/advanced*.csv')
team_adv_bat_y = pd.concat([pd.read_csv(fp) for fp in files], ignore_index=True)
# convert dates to timestamps
team_adv_bat_y['Season'] = pd.to_datetime(team_adv_bat_y['Season'], format='%Y').dt.year
team_adv_bat_y

,Season,Tm,PA,BB%,K%,BB/K,AVG,OBP,SLG,OPS,ISO,BABIP,wRC,wRAA,wOBA,wRC+
0,2002,ANA,6327,0.073020,0.127232,0.573913,0.282318,0.340714,0.432547,0.773260,0.150229,0.303239,810.797010,51.075588,0.335925,104.792946
1,2002,BAL,6096,0.074147,0.162894,0.455186,0.246403,0.308620,0.403023,0.711643,0.156620,0.271109,654.365338,-77.618507,0.310501,89.893491
2,2002,BOS,6332,0.086071,0.149084,0.577331,0.276596,0.345008,0.444149,0.789157,0.167553,0.302493,848.877642,88.555840,0.342839,107.079563
3,2002,CHW,6208,0.089401,0.153351,0.582983,0.268084,0.337555,0.449473,0.787028,0.181389,0.286822,826.725055,81.292687,0.342026,105.439602
4,2002,CLE,6099,0.088867,0.163961,0.542000,0.248755,0.321287,0.411580,0.732867,0.162825,0.270960,703.887840,-28.456233,0.320164,94.543848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2022,PHI,6077,0.078657,0.224288,0.350697,0.253275,0.316639,0.422125,0.738764,0.168850,0.298842,752.397511,57.514122,0.321607,105.812719
626,2022,PIT,5912,0.080514,0.253214,0.317969,0.222472,0.291193,0.363722,0.654915,0.141249,0.277238,577.138927,-98.877297,0.288710,84.226686
627,2022,STL,6165,0.087105,0.198865,0.438010,0.252183,0.325268,0.420124,0.745392,0.167940,0.288732,784.408701,79.462824,0.325941,113.657725
628,2022,SDP,6175,0.092955,0.214899,0.432555,0.240856,0.317894,0.381675,0.699569,0.140819,0.288547,698.402938,-7.686403,0.308077,101.810046


In [ ]:
# team batting batted ball stats per year
files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/Batting/per season/batted_balls*.csv')
team_bat_bat_y = pd.concat([pd.read_csv(fp) for fp in files], ignore_index=True)
# convert dates to timestamps
team_bat_bat_y['Season'] = pd.to_datetime(team_bat_bat_y['Season'], format='%Y').dt.year
team_bat_bat_y

,Season,Tm,PA,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%
0,2002,ANA,6327,1.036041,0.224862,0.394430,0.380709,0.109736,0.081764,0.059709,0.271845,0.433213,0.291215,0.275572,0.146155,0.640835,0.213009
1,2002,BAL,6096,1.045429,0.197951,0.409931,0.392118,0.147076,0.093697,0.058664,0.177083,0.481142,0.267059,0.251799,0.162852,0.610421,0.226728
2,2002,BOS,6332,1.216507,0.214997,0.430841,0.354162,0.118421,0.105861,0.067847,0.320000,0.459233,0.278348,0.262419,0.140730,0.622483,0.236787
3,2002,CHW,6208,1.022497,0.205830,0.401502,0.392668,0.123735,0.122047,0.051155,0.338843,0.467412,0.250592,0.281996,0.145932,0.619673,0.234395
4,2002,CLE,6099,1.243020,0.199321,0.443715,0.356965,0.125000,0.121827,0.055130,0.220930,0.465230,0.283493,0.251277,0.156841,0.623804,0.219355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2022,PHI,6077,1.167213,0.203230,0.429122,0.367647,0.112787,0.134426,0.075281,0.542857,0.389672,0.343772,0.266555,0.169974,0.526177,0.303849
626,2022,PIT,5912,1.219020,0.196452,0.441430,0.362118,0.100865,0.113833,0.063239,0.274510,0.404995,0.350412,0.244593,0.169928,0.534758,0.295314
627,2022,STL,6165,1.046690,0.195026,0.411669,0.393305,0.124113,0.116430,0.062676,0.444444,0.418981,0.342130,0.238889,0.182407,0.517130,0.300463
628,2022,SDP,6175,1.130185,0.197210,0.425926,0.376864,0.102744,0.097639,0.064935,0.369565,0.405328,0.344196,0.250476,0.181732,0.522598,0.295671


In [ ]:
# Team batting advanced stats per game
files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/Batting/per game/advanced*.csv')
team_adv_bat_g = pd.concat([pd.read_csv(fp) for fp in files], ignore_index=True)
# convert dates to timestamps
team_adv_bat_g['Date'] = pd.to_datetime(team_adv_bat_g['Date'])
team_adv_bat_g

,Date,Tm,PA,BB%,K%,BB/K,AVG,OBP,SLG,OPS,ISO,BABIP,wRC,wRAA,wOBA,wRC+
0,2002-03-31,ANA,33,0.060606,0.151515,0.400000,0.161290,0.212121,0.193548,0.405670,0.032258,0.192308,0.226965,-3.735546,0.188861,14.923632
1,2002-03-31,CLE,38,0.078947,0.236842,0.333333,0.314286,0.368421,0.428571,0.796992,0.114286,0.400000,5.416478,0.853586,0.353108,116.526569
2,2002-04-01,ATL,38,0.105263,0.105263,1.000000,0.235294,0.315789,0.500000,0.815789,0.264706,0.214286,5.320222,0.757331,0.350041,118.903758
3,2002-04-01,PHI,34,0.058824,0.294118,0.200000,0.161290,0.205882,0.322581,0.528463,0.161290,0.190476,1.310226,-2.772361,0.227190,36.342876
4,2002-04-01,BOS,52,0.250000,0.153846,1.625000,0.342105,0.500000,0.684211,1.184211,0.342105,0.333333,13.417794,7.173837,0.492940,207.710284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98969,2022-10-05,SFG,44,0.090909,0.181818,0.500000,0.324324,0.386364,0.594595,0.980958,0.270270,0.321429,8.522521,3.491277,0.409624,169.094419
98970,2022-10-05,LAA,33,0.030303,0.121212,0.250000,0.193548,0.242424,0.387097,0.629521,0.193548,0.192308,2.799228,-0.974205,0.272586,82.136999
98971,2022-10-05,OAK,31,0.064516,0.258065,0.250000,0.148148,0.193548,0.296296,0.489845,0.148148,0.150000,1.063355,-2.481385,0.208989,41.431650
98972,2022-10-05,DET,37,0.054054,0.270270,0.200000,0.228571,0.270270,0.257143,0.527413,0.028571,0.320000,2.137736,-2.093082,0.238538,56.439381


In [ ]:
# Team offensive batted ball stats per game
files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/Batting/per game/batted_balls*.csv')
team_bat_bat_g = pd.concat([pd.read_csv(fp) for fp in files], ignore_index=True)
# convert dates to timestamps
team_bat_bat_g['Date'] = pd.to_datetime(team_bat_bat_g['Date'])
team_bat_bat_g

,Date,Tm,PA,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%
0,2002-03-31,ANA,33,1.625000,0.192308,0.500000,0.307692,0.000000,0.000000,0.076923,0.0,0.384615,0.269231,0.346154,0.307692,0.538462,0.153846
1,2002-03-31,CLE,38,0.437500,0.115385,0.269231,0.615385,0.125000,0.062500,0.000000,0.0,0.346154,0.461538,0.192308,0.307692,0.384615,0.307692
2,2002-04-01,ATL,38,1.166667,0.133333,0.466667,0.400000,0.083333,0.166667,0.142857,0.0,0.433333,0.333333,0.233333,0.300000,0.366667,0.333333
3,2002-04-01,PHI,34,0.333333,0.090909,0.227273,0.681818,0.133333,0.066667,0.000000,0.0,0.272727,0.318182,0.409091,0.272727,0.363636,0.363636
4,2002-04-01,BOS,52,0.562500,0.166667,0.300000,0.533333,0.187500,0.250000,0.000000,0.0,0.354839,0.387097,0.258065,0.225806,0.419355,0.354839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98969,2022-10-05,SFG,44,1.875000,0.258065,0.483871,0.258065,0.000000,0.375000,0.000000,0.0,0.548387,0.387097,0.064516,0.161290,0.580645,0.258065
98970,2022-10-05,LAA,33,1.333333,0.222222,0.444444,0.333333,0.111111,0.111111,0.000000,0.0,0.407407,0.333333,0.259259,0.148148,0.629630,0.222222
98971,2022-10-05,OAK,31,0.900000,0.095238,0.428571,0.476190,0.100000,0.100000,0.000000,0.0,0.285714,0.619048,0.095238,0.095238,0.714286,0.190476
98972,2022-10-05,DET,37,0.615385,0.160000,0.320000,0.520000,0.076923,0.000000,0.250000,0.0,0.360000,0.280000,0.360000,0.440000,0.400000,0.160000


In [ ]:
# starting pitcher advanced stats per year
files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/Pitching/Starting Pitching/per season/advanced*.csv')
sp_adv_y = pd.concat([pd.read_csv(fp) for fp in files], ignore_index=True)
# convert dates to timestamps
sp_adv_y['Season'] = pd.to_datetime(sp_adv_y['Season'], format='%Y').dt.year
sp_adv_y

,Season,Name,Tm,IP,TBF,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,AVG,WHIP,BABIP,LOB%,FIP,xFIP,playerId
0,2002,Mike Hampton,COL,178.2,838,3.727612,4.583955,0.813187,1.208955,0.088305,0.108592,-0.020286,0.312757,1.785448,0.317757,0.653215,5.525507,5.236269,430
1,2002,Roger Clemens,NYY,180.0,768,9.600000,3.150000,3.047619,0.900000,0.250000,0.082031,0.167969,0.250000,1.305556,0.315574,0.682657,3.295408,3.317903,815
2,2002,Jon Lieber,CHC,141.0,582,5.553191,0.765957,7.250000,0.957447,0.149485,0.020619,0.128866,0.276673,1.170213,0.295503,0.703448,3.387606,3.604062,287
3,2002,Jamie Moyer,SEA,230.2,931,5.735549,1.950867,2.940000,1.092486,0.157895,0.053706,0.104189,0.230233,1.075145,0.243902,0.771350,4.032884,4.289525,1091
4,2002,Jeff Weaver,DET,121.2,509,5.547945,2.441096,2.272727,0.295890,0.147348,0.064833,0.082515,0.242950,1.191781,0.277635,0.698779,3.167554,4.250490,503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6860,2022,David Phelps,TOR,1.0,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,1.000000,0.333333,1.000000,6.112431,6.112431,6316
6861,2022,Elieser Hernandez,MIA,46.2,207,7.907143,2.507143,3.153846,2.700000,0.198068,0.062802,0.135266,0.265957,1.350000,0.264706,0.668103,6.283860,5.016802,16933
6862,2022,Craig Stammen,SDP,3.0,18,9.000000,9.000000,1.000000,3.000000,0.166667,0.166667,0.000000,0.400000,3.000000,0.454545,0.526316,8.445764,6.086257,7274
6863,2022,Jackson Stephens,ATL,3.0,15,0.000000,6.000000,0.000000,0.000000,0.000000,0.133333,-0.133333,0.384615,2.333333,0.384615,0.571429,5.112431,6.592800,14420


In [ ]:
# starting pitcher batted ball stats per year
files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/Pitching/Starting Pitching/per season/batted_balls*.csv')
sp_bat_y = pd.concat([pd.read_csv(fp) for fp in files], ignore_index=True)
# convert dates to timestamps
sp_bat_y['Season'] = pd.to_datetime(sp_bat_y['Season'], format='%Y').dt.year
sp_bat_y

,Season,Name,Tm,IP,TBF,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,playerId
0,2002,Jon Lieber,CHC,141.0,582,1.216049,0.222944,0.426407,0.350649,0.104938,0.092593,0.055838,0.150000,0.398340,0.282158,0.319502,0.192946,0.603734,0.203319,287
1,2002,Mike Hampton,COL,178.2,838,1.684492,0.233588,0.480916,0.285496,0.058824,0.128342,0.082540,0.454545,0.414414,0.309309,0.276276,0.151652,0.621622,0.226727,430
2,2002,Roger Clemens,NYY,180.0,768,1.274854,0.214141,0.440404,0.345455,0.093567,0.105263,0.059633,0.272727,0.407115,0.284585,0.308300,0.156436,0.649505,0.194059,815
3,2002,Kevin Appier,ANA,188.1,795,0.764925,0.185886,0.352840,0.461274,0.111940,0.085821,0.058537,0.636364,0.449324,0.305743,0.244932,0.155405,0.625000,0.219595,3
4,2002,Jamie Moyer,SEA,230.2,931,0.871711,0.199719,0.372714,0.427567,0.154605,0.092105,0.045283,0.285714,0.468966,0.246897,0.284138,0.186207,0.615172,0.198621,1091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6860,2022,Austin Gomber,COL,87.2,377,1.096154,0.215827,0.410072,0.374101,0.048077,0.125000,0.052632,0.500000,0.432624,0.365248,0.202128,0.152482,0.524823,0.322695,16561
6861,2022,Framber Valdez,HOU,201.1,827,4.159091,0.174545,0.665455,0.160000,0.090909,0.125000,0.057377,0.400000,0.432432,0.365766,0.201802,0.162162,0.536937,0.300901,17295
6862,2022,Craig Stammen,SDP,3.0,18,1.000000,0.333333,0.333333,0.333333,0.000000,0.250000,0.000000,0.000000,0.583333,0.333333,0.083333,0.083333,0.750000,0.166667,7274
6863,2022,Jackson Stephens,ATL,3.0,15,1.000000,0.538462,0.230769,0.230769,0.000000,0.000000,0.000000,0.000000,0.307692,0.076923,0.615385,0.230769,0.538462,0.230769,14420


In [ ]:
# starting pitcher advanced stats per game
files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/Pitching/Starting Pitching/per game/advanced*.csv')
sp_adv_g = pd.concat([pd.read_csv(fp) for fp in files], ignore_index=True)
# convert dates to timestamps
sp_adv_g['Date'] = pd.to_datetime(sp_adv_g['Date'])
sp_adv_g

,Date,Name,Tm,IP,TBF,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,AVG,WHIP,BABIP,LOB%,FIP,xFIP,playerId
0,2002-03-31,Jarrod Washburn,ANA,5.0,23,10.800000,1.800000,6.00,1.800000,0.260870,0.043478,0.217391,0.363636,1.800000,0.466667,0.526316,3.762074,3.946275,40
1,2002-03-31,Bartolo Colon,CLE,9.0,33,5.000000,2.000000,2.50,0.000000,0.151515,0.060606,0.090909,0.161290,0.777778,0.192308,1.000000,2.517630,3.755053,375
2,2002-04-01,Tom Glavine,ATL,6.1,26,9.947368,2.842105,3.50,1.421053,0.269231,0.076923,0.192308,0.208333,1.105263,0.250000,0.892857,3.751548,4.336581,90
3,2002-04-01,Robert Person,PHI,5.0,23,5.400000,7.200000,0.75,3.600000,0.130435,0.173913,-0.043478,0.157895,1.400000,0.071429,0.238095,9.362074,6.389435,966
4,2002-04-01,Pedro Martinez,BOS,3.0,23,12.000000,6.000000,2.00,0.000000,0.173913,0.086957,0.086957,0.500000,3.666667,0.600000,0.384615,4.295408,6.615575,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98959,2022-10-05,John Brebbia,SFG,1.0,5,9.000000,0.000000,1.00,0.000000,0.200000,0.000000,0.200000,0.400000,2.000000,0.500000,0.500000,1.112431,2.592800,12777
98960,2022-10-05,Shohei Ohtani,LAA,5.0,17,10.800000,1.800000,6.00,0.000000,0.352941,0.058824,0.294118,0.066667,0.400000,0.100000,0.500000,1.312431,2.496727,19755
98961,2022-10-05,Ken Waldichuk,OAK,7.0,25,5.142857,1.285714,4.00,0.000000,0.160000,0.040000,0.120000,0.130435,0.571429,0.157895,1.000000,2.826717,4.307086,27681
98962,2022-10-05,Marco Gonzales,SEA,7.0,31,6.428571,2.571429,2.50,0.000000,0.161290,0.064516,0.096774,0.275862,1.428571,0.333333,0.600000,2.541002,5.078779,15467


In [ ]:
# starting pitcher batted ball stats per game
files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/Pitching/Starting Pitching/per game/batted_balls*.csv')
sp_bat_g = pd.concat([pd.read_csv(fp) for fp in files], ignore_index=True)
# convert dates to timestamps
sp_bat_g['Date'] = pd.to_datetime(sp_bat_g['Date'])
sp_bat_g

,Date,Name,Tm,IP,TBF,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,playerId
0,2002-03-31,Jarrod Washburn,ANA,5.0,23,0.300000,0.187500,0.187500,0.625000,0.100000,0.100000,0.000000,0.0,0.437500,0.437500,0.125000,0.250000,0.375000,0.375000,40
1,2002-03-31,Bartolo Colon,CLE,9.0,33,1.625000,0.192308,0.500000,0.307692,0.000000,0.000000,0.076923,0.0,0.384615,0.269231,0.346154,0.307692,0.538462,0.153846,375
2,2002-04-01,Tom Glavine,ATL,6.1,26,0.250000,0.117647,0.176471,0.705882,0.166667,0.083333,0.000000,0.0,0.294118,0.294118,0.411765,0.235294,0.352941,0.411765,90
3,2002-04-01,Robert Person,PHI,5.0,23,0.875000,0.062500,0.437500,0.500000,0.000000,0.250000,0.142857,0.0,0.437500,0.375000,0.187500,0.375000,0.375000,0.250000,966
4,2002-04-01,Pedro Martinez,BOS,3.0,23,0.600000,0.428571,0.214286,0.357143,0.000000,0.000000,0.000000,1.0,0.133333,0.533333,0.333333,0.066667,0.600000,0.333333,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98959,2022-10-05,John Brebbia,SFG,1.0,5,3.000000,0.000000,0.750000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000,0.500000,0.250000,0.500000,0.250000,12777
98960,2022-10-05,Shohei Ohtani,LAA,5.0,17,1.250000,0.100000,0.500000,0.400000,0.250000,0.000000,0.000000,0.0,0.200000,0.700000,0.100000,0.100000,0.600000,0.300000,19755
98961,2022-10-05,Ken Waldichuk,OAK,7.0,25,1.142857,0.210526,0.421053,0.368421,0.142857,0.000000,0.000000,0.0,0.421053,0.315789,0.263158,0.210526,0.631579,0.157895,27681
98962,2022-10-05,Marco Gonzales,SEA,7.0,31,0.666667,0.166667,0.333333,0.500000,0.083333,0.000000,0.250000,0.0,0.375000,0.291667,0.333333,0.416667,0.416667,0.166667,15467


In [ ]:
# relief pitcher advanced stats per year
files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/Pitching/Relief Pitching/per season/advanced*.csv')
rp_adv_y = pd.concat([pd.read_csv(fp) for fp in files], ignore_index=True)
# convert dates to timestamps
rp_adv_y['Season'] = pd.to_datetime(rp_adv_y['Season'], format='%Y').dt.year
rp_adv_y

,Season,Tm,IP,TBF,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,AVG,WHIP,BABIP,LOB%,FIP,xFIP
0,2002,LAD,471.2,2001,7.441696,3.320141,2.241379,1.011307,0.194903,0.086957,0.107946,0.237374,1.265724,0.270666,0.763246,3.983983,3.995979
1,2002,STL,529.0,2188,6.533081,3.402646,1.920000,0.816635,0.175503,0.091408,0.084095,0.233091,1.224953,0.259740,0.772453,3.914815,4.122069
2,2002,CIN,574.0,2428,7.087108,3.418118,2.073395,0.815331,0.186161,0.089786,0.096376,0.247427,1.301394,0.283591,0.727585,3.829670,4.097426
3,2002,BOS,453.0,1943,7.251656,3.238411,2.239264,0.973510,0.187854,0.083891,0.103963,0.250731,1.306843,0.285071,0.709921,4.054790,4.277335
4,2002,MIL,505.1,2173,7.034960,4.220976,1.666667,1.086412,0.181776,0.109066,0.072711,0.237480,1.341689,0.260810,0.750487,4.511547,4.531814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2022,SFG,650.0,2808,8.058462,3.336923,2.414938,0.900000,0.207265,0.085826,0.121439,0.259436,1.375385,0.309637,0.705097,3.830893,4.047702
626,2022,CIN,613.2,2698,9.283542,4.341119,2.138514,1.202607,0.234618,0.109711,0.124907,0.247746,1.422596,0.300912,0.708573,4.438884,4.354234
627,2022,LAA,592.1,2485,8.402364,3.433877,2.446903,1.230726,0.222535,0.090946,0.131590,0.226697,1.227349,0.262993,0.701372,4.309392,4.186158
628,2022,STL,571.1,2390,7.986581,3.386814,2.358140,0.929405,0.212134,0.089958,0.122176,0.233803,1.247958,0.276448,0.752226,3.919315,4.001935


In [ ]:
# relief pitcher batted ball stats per year
files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/Pitching/Relief Pitching/per season/batted_balls*.csv')
rp_bat_y = pd.concat([pd.read_csv(fp) for fp in files], ignore_index=True)
# convert dates to timestamps
rp_bat_y['Season'] = pd.to_datetime(rp_bat_y['Season'], format='%Y').dt.year
rp_bat_y

,Season,Tm,IP,TBF,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%
0,2002,LAD,471.2,2001,1.226453,0.200719,0.440288,0.358993,0.128257,0.106212,0.052288,0.233333,0.364789,0.285211,0.350000,0.177590,0.620860,0.201550
1,2002,STL,529.0,2188,1.303605,0.215255,0.444085,0.340659,0.121442,0.091082,0.064047,0.075000,0.431632,0.296786,0.271582,0.160151,0.622951,0.216898
2,2002,CIN,574.0,2428,1.214765,0.212880,0.431723,0.355397,0.127517,0.087248,0.074586,0.232143,0.410271,0.316215,0.273514,0.167436,0.606813,0.225751
3,2002,BOS,453.0,1943,1.052830,0.199411,0.410596,0.389993,0.126415,0.092453,0.051971,0.181818,0.420710,0.278783,0.300507,0.145547,0.647357,0.207096
4,2002,MIL,505.1,2173,1.003466,0.208761,0.396304,0.394935,0.145581,0.105719,0.058722,0.125000,0.405405,0.290046,0.304548,0.170297,0.617822,0.211881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2022,SFG,650.0,2808,1.339339,0.196493,0.460031,0.343476,0.091592,0.097598,0.079596,0.260870,0.401631,0.338940,0.259429,0.185015,0.536188,0.278797
626,2022,CIN,613.2,2698,1.011679,0.194623,0.405026,0.400351,0.121168,0.119708,0.069264,0.250000,0.402432,0.355530,0.242038,0.169079,0.536190,0.294731
627,2022,LAA,592.1,2485,1.001511,0.200362,0.400121,0.399517,0.089124,0.122357,0.064857,0.363636,0.415128,0.349017,0.235855,0.151876,0.541989,0.306135
628,2022,STL,571.1,2390,1.403636,0.189951,0.473039,0.337010,0.094545,0.107273,0.053109,0.142857,0.402795,0.345687,0.251519,0.176792,0.538275,0.284933


In [ ]:
# relief pitcher advanced stats per game
files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/Pitching/Relief Pitching/per game/advanced*.csv')
rp_adv_g = pd.concat([pd.read_csv(fp) for fp in files], ignore_index=True)
# convert dates to timestamps
rp_adv_g['Date'] = pd.to_datetime(rp_adv_g['Date'])
rp_adv_g

,Date,Tm,IP,TBF,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,AVG,WHIP,BABIP,LOB%,FIP,xFIP
0,2002-03-31,ANA,4.0,15,6.750,4.500,1.500000,0.0,0.200000,0.133333,0.066667,0.230769,1.250000,0.300000,0.800000,2.962074,5.050225
1,2002-04-01,ATL,2.2,8,10.125,0.000,3.000000,0.0,0.375000,0.000000,0.375000,0.000000,0.000000,0.000000,1.000000,0.712074,2.278187
2,2002-04-01,PHI,3.0,15,3.000,0.000,1.000000,0.0,0.066667,0.000000,0.066667,0.333333,1.666667,0.357143,0.800000,2.295408,4.151542
3,2002-04-01,BOS,6.0,29,4.500,6.000,0.750000,0.0,0.103448,0.137931,-0.034483,0.250000,1.500000,0.250000,0.636364,4.962074,6.586192
4,2002-04-01,TOR,6.2,36,6.750,13.500,0.500000,0.0,0.138889,0.277778,-0.138889,0.280000,2.550000,0.333333,0.705882,5.962074,8.050225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96506,2022-10-05,SFG,8.0,26,12.375,1.125,11.000000,0.0,0.423077,0.038462,0.384615,0.080000,0.375000,0.142857,1.000000,0.737431,1.847708
96507,2022-10-05,LAA,3.0,14,6.000,3.000,2.000000,3.0,0.142857,0.071429,0.071429,0.250000,1.333333,0.200000,0.769231,7.112431,5.739836
96508,2022-10-05,OAK,2.0,8,0.000,0.000,0.000000,4.5,0.000000,0.000000,0.000000,0.375000,1.500000,0.285714,0.625000,9.612431,4.592800
96509,2022-10-05,DET,2.2,13,3.375,10.125,0.333333,0.0,0.076923,0.230769,-0.153846,0.200000,1.875000,0.222222,0.800000,5.737431,7.402847


In [ ]:
# relief pitcher batted ball stats per game
files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/Pitching/Relief Pitching/per game/batted_balls*.csv')
rp_bat_g = pd.concat([pd.read_csv(fp) for fp in files], ignore_index=True)
# convert dates to timestamps
rp_bat_g['Date'] = pd.to_datetime(rp_bat_g['Date'])
rp_bat_g

,Date,Tm,IP,TBF,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%
0,2002-03-31,ANA,4.0,15,0.666667,0.000000,0.400000,0.600000,0.166667,0.000000,0.000000,0.0,0.200000,0.500000,0.300000,0.400000,0.400000,0.200000
1,2002-04-01,ATL,2.2,8,0.666667,0.000000,0.400000,0.600000,0.000000,0.000000,0.000000,0.0,0.200000,0.400000,0.400000,0.400000,0.400000,0.200000
2,2002-04-01,PHI,3.0,15,1.750000,0.214286,0.500000,0.285714,0.250000,0.000000,0.142857,0.0,0.428571,0.285714,0.285714,0.214286,0.357143,0.428571
3,2002-04-01,BOS,6.0,29,1.142857,0.210526,0.421053,0.368421,0.142857,0.000000,0.250000,0.0,0.500000,0.350000,0.150000,0.150000,0.800000,0.050000
4,2002-04-01,TOR,6.2,36,0.500000,0.250000,0.250000,0.500000,0.300000,0.000000,0.000000,0.0,0.285714,0.380952,0.333333,0.285714,0.476190,0.238095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96506,2022-10-05,SFG,8.0,26,1.166667,0.071429,0.500000,0.428571,0.166667,0.000000,0.000000,0.0,0.285714,0.357143,0.357143,0.214286,0.571429,0.214286
96507,2022-10-05,LAA,3.0,14,0.666667,0.090909,0.363636,0.545455,0.000000,0.166667,0.000000,0.0,0.363636,0.545455,0.090909,0.090909,0.818182,0.090909
96508,2022-10-05,OAK,2.0,8,2.000000,0.250000,0.500000,0.250000,0.000000,0.500000,0.000000,0.0,0.375000,0.375000,0.250000,0.000000,0.625000,0.375000
96509,2022-10-05,DET,2.2,13,2.000000,0.000000,0.666667,0.333333,0.000000,0.000000,0.166667,0.0,0.333333,0.555556,0.111111,0.222222,0.555556,0.222222


In [ ]:
# team defensive stats per year
team_def_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/team_def_data.csv')
# rename Season column to get rid of ghost character in column name
# team_def_data = team_def_data.rename(columns={"﻿Season":"Season"})
# convert dates to timestamps
team_def_data['Season'] = pd.to_datetime(team_def_data['Season'], format='%Y').dt.year
# convert team column data to object type so it is supported by cudf apply func.
# team_def_data['Team'] = team_def_data['Team'].astype('object')
team_def_data

,Season,Team,Inn,rSZ,rCERA,rSB,rGDP,rARM,rGFP,rPM,...,ARM,DPR,RngR,ErrR,UZR,UZR/150,FRM,OAA,RAA,Def
0,2009,Mariners,13074.0,0,0,12,3,0,-6,66,...,-0.2,6.1,92.3,-12.2,86.0,11.8,-35.1,NaN,NaN,96.9
1,2003,Mariners,12969.0,0,-2,2,6,-1,0,31,...,-0.4,4.5,60.5,11.5,76.1,10.8,NaN,NaN,NaN,83.1
2,2007,Royals,12936.0,0,-4,-1,7,-2,0,38,...,9.3,3.1,63.2,-3.3,72.3,11.4,NaN,NaN,NaN,75.3
3,2011,Diamondbacks,12990.0,12,2,5,1,-6,5,37,...,-4.0,-0.8,58.6,6.9,60.6,7.2,18.4,NaN,NaN,73.6
4,2008,Rays,13119.0,0,-4,0,3,2,-2,9,...,5.4,3.4,65.9,-2.1,72.6,11.4,-21.7,NaN,NaN,71.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2011,Mets,13032.0,-6,-1,-7,-10,1,-15,-31,...,-7.6,-6.9,-33.2,-14.2,-61.9,-9.8,-15.2,NaN,NaN,-72.5
626,2006,Yankees,12993.0,0,-4,3,1,-8,3,-34,...,-6.5,-1.5,-62.6,-9.4,-80.1,-11.8,NaN,NaN,NaN,-75.1
627,2004,Yankees,12993.0,0,-4,-4,-2,-3,-5,-54,...,-6.7,-2.4,-70.6,10.3,-69.4,-10.3,NaN,NaN,NaN,-78.4
628,2003,Blue Jays,12915.0,0,0,-8,-1,-12,0,-12,...,-14.5,-5.9,-43.2,-1.8,-65.5,-11.8,NaN,NaN,NaN,-84.5


In [ ]:
team_abbr = {'Mariners':'SEA', 'Royals':'KCR', 'Diamondbacks':'ARI',
                 'Rays':'TBR', 'Rangers':'TEX', 'Phillies':'PHI',
                 'Devil Rays':'TBD', 'Twins':'MIN', 'Reds':'CIN',
                 'Athletics':'OAK', 'Astros':'HOU', 'Giants':'SFG',
                 'Red Sox':'BOS', 'Braves':'ATL', 'Dodgers':'LAD',
                 'Padres':'SDP', 'Cubs':'CHC', 'Tigers':'DET',
                 'Cardinals':'STL', 'Orioles':'BAL', 'Angels':['ANA','LAA'],
                 'Yankees':'NYY', 'Indians':'CLE', 'Nationals':'WSN',
                 'Blue Jays':'TOR', 'Brewers':'MIL', 'White Sox':'CHW',
                 'Mets':'NYM', 'Marlins':['FLA','MIA'], 'Pirates':'PIT',
                 'Guardians':'CLE', 'Cleveland':'CLE', 'Rockies':'COL',
                 'Expos':'MON',
                 }

def get_team_abbreviation(row):
    team_name = row['Team']
    year = row['Season']
    if isinstance(team_abbr[team_name], str):
        return team_abbr[team_name]
    elif team_name == 'Angels' and year < 2005:
        return team_abbr[team_name][0]
    elif team_name == 'Marlins' and year < 2012:
        return team_abbr[team_name][0]
    else:
        return team_abbr[team_name][1]

In [ ]:
# add a new column to the dataframe with the team abbreviations
team_def_data['team_abbr'] = team_def_data.apply(get_team_abbreviation, axis=1)
# drop Team column and rename team_abbr to Team
team_def_data = team_def_data.drop(columns='Team').rename(columns={'team_abbr':'Team'})
team_def_data = team_def_data.drop(columns=['rTS','FRM','OAA','RAA'])
team_def_data

,Season,Inn,rSZ,rCERA,rSB,rGDP,rARM,rGFP,rPM,DRS,ARM,DPR,RngR,ErrR,UZR,UZR/150,Def,Team
0,2009,13074.0,0,0,12,3,0,-6,66,80,-0.2,6.1,92.3,-12.2,86.0,11.8,96.9,SEA
1,2003,12969.0,0,-2,2,6,-1,0,31,41,-0.4,4.5,60.5,11.5,76.1,10.8,83.1,SEA
2,2007,12936.0,0,-4,-1,7,-2,0,38,43,9.3,3.1,63.2,-3.3,72.3,11.4,75.3,KCR
3,2011,12990.0,12,2,5,1,-6,5,37,57,-4.0,-0.8,58.6,6.9,60.6,7.2,73.6,ARI
4,2008,13119.0,0,-4,0,3,2,-2,9,15,5.4,3.4,65.9,-2.1,72.6,11.4,71.1,TBR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2011,13032.0,-6,-1,-7,-10,1,-15,-31,-60,-7.6,-6.9,-33.2,-14.2,-61.9,-9.8,-72.5,NYM
626,2006,12993.0,0,-4,3,1,-8,3,-34,-39,-6.5,-1.5,-62.6,-9.4,-80.1,-11.8,-75.1,NYY
627,2004,12993.0,0,-4,-4,-2,-3,-5,-54,-79,-6.7,-2.4,-70.6,10.3,-69.4,-10.3,-78.4,NYY
628,2003,12915.0,0,0,-8,-1,-12,0,-12,-47,-14.5,-5.9,-43.2,-1.8,-65.5,-11.8,-84.5,TOR


In [ ]:
# check for NaN values in the database
team_def_data.isna().sum()

Season     0
Inn        0
rSZ        0
rCERA      0
rSB        0
rGDP       0
rARM       0
rGFP       0
rPM        0
DRS        0
ARM        0
DPR        0
RngR       0
ErrR       0
UZR        0
UZR/150    0
Def        0
Team       0
dtype: int64

In [ ]:
# save team_def_data as team_def_data_formatted.csv
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/team_def_data_formatted.csv')
team_def_data.to_csv(filepath, index=False)

In [ ]:
# load team_def_data_formatted
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/team_def_data_formatted.csv')
team_def_data_f = pd.read_csv(filepath)

In [ ]:
team_def_data_f

,Season,Inn,rSZ,rCERA,rSB,rGDP,rARM,rGFP,rPM,DRS,ARM,DPR,RngR,ErrR,UZR,UZR/150,Def,Team
0,2009,13074.0,0,0,12,3,0,-6,66,80,-0.2,6.1,92.3,-12.2,86.0,11.8,96.9,SEA
1,2003,12969.0,0,-2,2,6,-1,0,31,41,-0.4,4.5,60.5,11.5,76.1,10.8,83.1,SEA
2,2007,12936.0,0,-4,-1,7,-2,0,38,43,9.3,3.1,63.2,-3.3,72.3,11.4,75.3,KCR
3,2011,12990.0,12,2,5,1,-6,5,37,57,-4.0,-0.8,58.6,6.9,60.6,7.2,73.6,ARI
4,2008,13119.0,0,-4,0,3,2,-2,9,15,5.4,3.4,65.9,-2.1,72.6,11.4,71.1,TBR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2011,13032.0,-6,-1,-7,-10,1,-15,-31,-60,-7.6,-6.9,-33.2,-14.2,-61.9,-9.8,-72.5,NYM
626,2006,12993.0,0,-4,3,1,-8,3,-34,-39,-6.5,-1.5,-62.6,-9.4,-80.1,-11.8,-75.1,NYY
627,2004,12993.0,0,-4,-4,-2,-3,-5,-54,-79,-6.7,-2.4,-70.6,10.3,-69.4,-10.3,-78.4,NYY
628,2003,12915.0,0,0,-8,-1,-12,0,-12,-47,-14.5,-5.9,-43.2,-1.8,-65.5,-11.8,-84.5,TOR


In [ ]:
# get every game from 2002 to 2022 and pitchers used to extract starting pitchers 
games = pd.DataFrame()
for year in range(2002,2023):
    print(f'year {year}')
    for team in sp_adv_y['Team'].unique():
        print(f'---team {team}')
        if team == 'WSN' and year < 2005:
            continue
        elif team == 'MON' and year > 2004:
            continue
        elif team == 'TBR' and year < 2008:
            continue
        elif team == 'TBD' and year > 2007:
            continue
        elif team == 'LAA' and year < 2005:
            continue
        elif team == 'ANA' and year > 2004:
            continue
        elif team == 'MIA' and year < 2012:
            continue
        elif team == 'FLA' and year > 2011:
            continue
        else:
            db = pybaseball.team_game_logs(year, team, 'pitching')
            db['Team'] = team
            db['Year'] = year
            games = pd.concat([games,db])

filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/raw_games.csv')
games.to_csv(filepath, index=False)

In [ ]:
raw_games = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/raw_games.csv')

In [ ]:
raw_games

,Game,Date,Home,Opp,Rslt,IP,H,R,ER,UER,...,IBB,SH,SF,ROE,GDP,NumPlayers,Umpire,PitchersUsed,Team,Year
0,1,Apr 1,False,STL,"L,2-10",8.0,14,10,10,0,...,1,0,1,0,0,6,Larry Young,"M.Hampton (99-17-L), D.Reyes (99), M.James (99...",COL,2002
1,2,Apr 3,False,STL,"W,6-3",9.0,9,3,3,0,...,0,0,0,2,0,5,Bill Hohn,"D.Neagle (99-61-W), M.James (1), K.Mercker (99...",COL,2002
2,3,Apr 4,False,STL,"W,6-1",9.0,8,1,1,0,...,0,0,1,0,1,3,Fieldin Culbreth,"J.Thomson (99-62-W), K.Mercker (0), M.James (0)",COL,2002
3,4,Apr 5,False,LAD,"L,0-9",8.0,11,9,9,0,...,0,1,0,0,0,3,Rick Reed,"S.Chacon (99-15-L), D.Reyes (3), R.White (3)",COL,2002
4,5,Apr 6,False,LAD,"L,2-9",8.0,9,9,4,5,...,0,0,0,1,1,5,Tim Tschida,"J.Jennings (99-51-L), M.James (1), K.Mercker (...",COL,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98969,158,Oct 1,False,MIL,"W,4-3",9.0,7,3,3,0,...,0,0,0,0,0,8,Hunter Wendelstedt,"E.Cabrera (5-64), A.Nardi (2), J.Brigham (1-H)...",MIA,2022
98970,159,Oct 2,False,MIL,"W,4-3",12.0,7,3,2,1,...,2,2,0,0,2,6,Chad Whitson,"P.López (4-77), R.Bleier (0-H), D.Floro (2-BSv...",MIA,2022
98971,160,Oct 3,True,ATL,"W,4-0",9.0,4,0,0,0,...,0,0,0,1,1,4,Manny Gonzalez,"J.Luzardo (4-75-W), J.Brigham (1), B.Hoeing (3...",MIA,2022
98972,161,Oct 4,True,ATL,"L,1-2",9.0,8,2,2,0,...,1,1,0,0,2,5,Jordan Baker,"B.Garrett (4-46-L), H.Brazoban (1), T.Nance (1...",MIA,2022


In [ ]:
# create a copy of the raw_games database called games which will be used to clean up the data
games = raw_games.copy()

In [ ]:
games

,Game,Date,Home,Opp,Rslt,IP,H,R,ER,UER,...,IBB,SH,SF,ROE,GDP,NumPlayers,Umpire,PitchersUsed,Team,Year
0,1,Apr 1,False,STL,"L,2-10",8.0,14,10,10,0,...,1,0,1,0,0,6,Larry Young,"M.Hampton (99-17-L), D.Reyes (99), M.James (99...",COL,2002
1,2,Apr 3,False,STL,"W,6-3",9.0,9,3,3,0,...,0,0,0,2,0,5,Bill Hohn,"D.Neagle (99-61-W), M.James (1), K.Mercker (99...",COL,2002
2,3,Apr 4,False,STL,"W,6-1",9.0,8,1,1,0,...,0,0,1,0,1,3,Fieldin Culbreth,"J.Thomson (99-62-W), K.Mercker (0), M.James (0)",COL,2002
3,4,Apr 5,False,LAD,"L,0-9",8.0,11,9,9,0,...,0,1,0,0,0,3,Rick Reed,"S.Chacon (99-15-L), D.Reyes (3), R.White (3)",COL,2002
4,5,Apr 6,False,LAD,"L,2-9",8.0,9,9,4,5,...,0,0,0,1,1,5,Tim Tschida,"J.Jennings (99-51-L), M.James (1), K.Mercker (...",COL,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98969,158,Oct 1,False,MIL,"W,4-3",9.0,7,3,3,0,...,0,0,0,0,0,8,Hunter Wendelstedt,"E.Cabrera (5-64), A.Nardi (2), J.Brigham (1-H)...",MIA,2022
98970,159,Oct 2,False,MIL,"W,4-3",12.0,7,3,2,1,...,2,2,0,0,2,6,Chad Whitson,"P.López (4-77), R.Bleier (0-H), D.Floro (2-BSv...",MIA,2022
98971,160,Oct 3,True,ATL,"W,4-0",9.0,4,0,0,0,...,0,0,0,1,1,4,Manny Gonzalez,"J.Luzardo (4-75-W), J.Brigham (1), B.Hoeing (3...",MIA,2022
98972,161,Oct 4,True,ATL,"L,1-2",9.0,8,2,2,0,...,1,1,0,0,2,5,Jordan Baker,"B.Garrett (4-46-L), H.Brazoban (1), T.Nance (1...",MIA,2022


In [ ]:
# remove game num from date
def remove_game_num(date):
    tags = ['(1)','(2)','susp']
    for tag in tags:
        date = date.replace(tag,'')
    return date

In [ ]:
def home_away_score(row):
    scores = row['Rslt'].split(',')[1].split('-')
    if row['Home'] == True:
        row['Home Score'] = scores[0]
        row['Away Score'] = scores[1]
    else:
        row['Home Score'] = scores[1]
        row['Away Score'] = scores[0]

    return row

In [ ]:
# create starting pitcher columnn for each game in input
def starting_pitcher(row):
    row['Starting Pitcher'] = row['PitchersUsed'].split(" (")[0]
    return row

In [ ]:
# write a func that removes punctuation from player names using str.replace() and the pattern
def remove_punctuation(text):
    pattern = r'[^\w\s]'
    text = text.replace(pattern, '')
    text = text.replace('ñ', 'n')
    text = text.replace('á', 'a')
    text = text.replace('é', 'e')
    text = text.replace('í', 'i')
    text = text.replace('ó', 'o')
    text = text.replace('ú', 'u')
    text = text.replace("'", "")
    text = text.replace('.', '')
    text = text.replace(' ', '')
    return text

In [ ]:
# write a function that can take in the raw games database and return cleaned up and formatted database of games
def format_raw_games(raw_games):
    # remove game num from date
    raw_games['Date'] = raw_games['Date'].apply(remove_game_num)
    # combine date and year column into a single date column
    raw_games['Date'] = raw_games[['Date','Year']].astype(str).apply(', '.join, axis=1)
    # turn date into datetime object
    raw_games['Date'] = pd.to_datetime(raw_games['Date'])
    # assign home and away teams
    raw_games['Home Team'] = raw_games['Team'].where(raw_games['Home'], raw_games['Opp'])
    raw_games['Away Team'] = raw_games['Opp'].where(raw_games['Home'], raw_games['Team'])
    # create home and away score columns
    raw_games = raw_games.apply(home_away_score, axis=1)
    # create starting pitcher columnn for each game in input
    raw_games = raw_games.apply(starting_pitcher, axis=1)
    # remove punctuation from players names
    raw_games['Starting Pitcher'] = raw_games['Starting Pitcher'].apply(remove_punctuation)
    # rename 'Year' column to 'Season'
    raw_games = raw_games.rename(columns={'Year':'Season'})
    # create Game ID column 
    raw_games['Game ID'] = raw_games['Date'].astype(str) + '_' + \
                   raw_games['Home Team'].str.strip() + '_' + \
                   raw_games['Away Team'].str.strip() + '_' + \
                   raw_games['Home Score'].astype(str) + '-' + \
                   raw_games['Away Score'].astype(str)
    # drop uneeded data
    raw_games = raw_games[['Game ID','Date','Season','Home','Team','Home Team','Away Team','Home Score','Away Score','Starting Pitcher']]
    return raw_games

In [ ]:
games = format_raw_games(raw_games)

In [ ]:
games

,Game ID,Date,Season,Home,Team,Home Team,Away Team,Home Score,Away Score,Starting Pitcher
0,2002-04-01_STL_COL_10-2,2002-04-01,2002,False,COL,STL,COL,10,2,MHampton
1,2002-04-03_STL_COL_3-6,2002-04-03,2002,False,COL,STL,COL,3,6,DNeagle
2,2002-04-04_STL_COL_1-6,2002-04-04,2002,False,COL,STL,COL,1,6,JThomson
3,2002-04-05_LAD_COL_9-0,2002-04-05,2002,False,COL,LAD,COL,9,0,SChacon
4,2002-04-06_LAD_COL_9-2,2002-04-06,2002,False,COL,LAD,COL,9,2,JJennings
...,...,...,...,...,...,...,...,...,...,...
98969,2022-10-01_MIL_MIA_3-4,2022-10-01,2022,False,MIA,MIL,MIA,3,4,ECabrera
98970,2022-10-02_MIL_MIA_3-4,2022-10-02,2022,False,MIA,MIL,MIA,3,4,PLopez
98971,2022-10-03_MIA_ATL_4-0,2022-10-03,2022,True,MIA,MIA,ATL,4,0,JLuzardo
98972,2022-10-04_MIA_ATL_1-2,2022-10-04,2022,True,MIA,MIA,ATL,1,2,BGarrett


In [ ]:
# save game pitchers database and starting pitchers.csv
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/games.csv')
games.to_csv(filepath, index=False)

In [ ]:
games = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/games.csv')

In [ ]:
# write a function that creates target variables based on desired split

# Create a new column that calculates the home team's ouctome margin
games['Outcome'] = games['Home Score'] - games['Away Score']

# Create the target variable categories
# bins = pd.IntervalIndex.from_tuples([(-float('inf'), -5), (-4, -2), (-1,-1), (1,1), (2, 4), (5, float('inf'))])
games['Target Variable'] = pd.cut(games['Outcome'], bins=[-float('inf'), -5, -3, -1, 2, 4, float('inf')], labels=['<=-5', '-3 to -4', '-1 to -2', '1 to 2', '3 to 4', '>=5'])
dummies = pd.get_dummies(games['Target Variable'], prefix='Score')
games = pd.concat([games,dummies], axis=1)

In [ ]:
games

,Game ID,Date,Season,Home,Team,Home Team,Away Team,Home Score,Away Score,Starting Pitcher,Outcome,Target Variable,Score_<=-5,Score_-3 to -4,Score_-1 to -2,Score_1 to 2,Score_3 to 4,Score_>=5
0,2002-04-01_STL_COL_10-2,2002-04-01,2002,False,COL,STL,COL,10,2,MHampton,8,>=5,0,0,0,0,0,1
1,2002-04-03_STL_COL_3-6,2002-04-03,2002,False,COL,STL,COL,3,6,DNeagle,-3,-3 to -4,0,1,0,0,0,0
2,2002-04-04_STL_COL_1-6,2002-04-04,2002,False,COL,STL,COL,1,6,JThomson,-5,<=-5,1,0,0,0,0,0
3,2002-04-05_LAD_COL_9-0,2002-04-05,2002,False,COL,LAD,COL,9,0,SChacon,9,>=5,0,0,0,0,0,1
4,2002-04-06_LAD_COL_9-2,2002-04-06,2002,False,COL,LAD,COL,9,2,JJennings,7,>=5,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98969,2022-10-01_MIL_MIA_3-4,2022-10-01,2022,False,MIA,MIL,MIA,3,4,ECabrera,-1,-1 to -2,0,0,1,0,0,0
98970,2022-10-02_MIL_MIA_3-4,2022-10-02,2022,False,MIA,MIL,MIA,3,4,PLopez,-1,-1 to -2,0,0,1,0,0,0
98971,2022-10-03_MIA_ATL_4-0,2022-10-03,2022,True,MIA,MIA,ATL,4,0,JLuzardo,4,3 to 4,0,0,0,0,1,0
98972,2022-10-04_MIA_ATL_1-2,2022-10-04,2022,True,MIA,MIA,ATL,1,2,BGarrett,-1,-1 to -2,0,0,1,0,0,0


In [ ]:
# save games w/targets as games_formatted.csv
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/games_formatted.csv')
games.to_csv(filepath, index=False)

In [ ]:
# import games_formatted csv
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/games_formatted.csv')
games_f = pd.read_csv(filepath)

In [ ]:
games_f

,Game ID,Date,Season,Home,Team,Home Team,Away Team,Home Score,Away Score,Starting Pitcher,Outcome,Target Variable,Score_<=-5,Score_-3 to -4,Score_-1 to -2,Score_1 to 2,Score_3 to 4,Score_>=5
0,2002-04-01_STL_COL_10-2,2002-04-01,2002,False,COL,STL,COL,10,2,MHampton,8,>=5,0,0,0,0,0,1
1,2002-04-03_STL_COL_3-6,2002-04-03,2002,False,COL,STL,COL,3,6,DNeagle,-3,-3 to -4,0,1,0,0,0,0
2,2002-04-04_STL_COL_1-6,2002-04-04,2002,False,COL,STL,COL,1,6,JThomson,-5,<=-5,1,0,0,0,0,0
3,2002-04-05_LAD_COL_9-0,2002-04-05,2002,False,COL,LAD,COL,9,0,SChacon,9,>=5,0,0,0,0,0,1
4,2002-04-06_LAD_COL_9-2,2002-04-06,2002,False,COL,LAD,COL,9,2,JJennings,7,>=5,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98969,2022-10-01_MIL_MIA_3-4,2022-10-01,2022,False,MIA,MIL,MIA,3,4,ECabrera,-1,-1 to -2,0,0,1,0,0,0
98970,2022-10-02_MIL_MIA_3-4,2022-10-02,2022,False,MIA,MIL,MIA,3,4,PLopez,-1,-1 to -2,0,0,1,0,0,0
98971,2022-10-03_MIA_ATL_4-0,2022-10-03,2022,True,MIA,MIA,ATL,4,0,JLuzardo,4,3 to 4,0,0,0,0,1,0
98972,2022-10-04_MIA_ATL_1-2,2022-10-04,2022,True,MIA,MIA,ATL,1,2,BGarrett,-1,-1 to -2,0,0,1,0,0,0


In [ ]:
def format_pitch_db(df, period, names=False):
    # if pull% is a column in the database, drop pull%, cent%, and oppo% columns
    if 'Pull%' in df.columns:
        df = df.drop(columns=['Pull%','Cent%', 'Oppo%'])
    # change 'Tm' column to 'Team'
    df = df.rename(columns={'Tm':'Team'})
    # check period
    if period == 'season':
        # set prefix to be '/yr'
        prefix = r'/yr '
        df['Season'] = pd.to_datetime(df['Season'], format='%Y').dt.year
    elif period == 'game':
        # set prefix to be '/g'
        prefix = r'/g '
        df['Date'] = pd.to_datetime(df['Date'])
    if names:
        # modify prefix to by 'sp'+prefix
        prefix = 'sp'+prefix
        # columns to be excluded from prefix being added
        if period == 'season':
            exclude_cols = ['Season','Team','Starting Pitcher']
        elif period == 'game':
            exclude_cols = ['Date','Team','Starting Pitcher']
        # first create a first and last name column
        df[['first name', 'last name']] = df['Name'].str.split(n=1, expand=True)
        # combine first and last name columns to be first initial of first name then last name
        df['Starting Pitcher'] = df['first name'].str[0] + df['last name']
        # remove punctuation from starting pitcher column
        df['Starting Pitcher'] = df['Starting Pitcher'].apply(remove_punctuation)
        # drop columns
        df = df.drop(columns=['Name','IP','TBF','playerId','first name','last name'])
        # Add prefix to all columns except excluded columns
        df = pd.concat([df[exclude_cols], df.drop(columns=exclude_cols).add_prefix(prefix)], axis=1)
    else:
        # modify prefix to by 'rp'+prefix
        prefix = 'rp'+prefix
        # columns to be excluded from prefix being added
        if period == 'season':
            exclude_cols = ['Season','Team']
        elif period == 'game':
            exclude_cols = ['Date','Team']
        df = df.drop(columns=['IP','TBF'])
        # Add prefix to all columns except excluded columns
        df = pd.concat([df[exclude_cols], df.drop(columns=exclude_cols).add_prefix(prefix)], axis=1)
    
    return df

In [ ]:
# format starting pitcher advanced stats per year database
sp_adv_y_f = format_pitch_db(sp_adv_y, period='season', names=True)

In [ ]:
sp_adv_y_f

,Season,Team,Starting Pitcher,sp/yr K/9,sp/yr BB/9,sp/yr K/BB,sp/yr HR/9,sp/yr K%,sp/yr BB%,sp/yr K-BB%,sp/yr AVG,sp/yr WHIP,sp/yr BABIP,sp/yr LOB%,sp/yr FIP,sp/yr xFIP
0,2002,COL,MHampton,3.727612,4.583955,0.813187,1.208955,0.088305,0.108592,-0.020286,0.312757,1.785448,0.317757,0.653215,5.525507,5.236269
1,2002,NYY,RClemens,9.600000,3.150000,3.047619,0.900000,0.250000,0.082031,0.167969,0.250000,1.305556,0.315574,0.682657,3.295408,3.317903
2,2002,CHC,JLieber,5.553191,0.765957,7.250000,0.957447,0.149485,0.020619,0.128866,0.276673,1.170213,0.295503,0.703448,3.387606,3.604062
3,2002,SEA,JMoyer,5.735549,1.950867,2.940000,1.092486,0.157895,0.053706,0.104189,0.230233,1.075145,0.243902,0.771350,4.032884,4.289525
4,2002,DET,JWeaver,5.547945,2.441096,2.272727,0.295890,0.147348,0.064833,0.082515,0.242950,1.191781,0.277635,0.698779,3.167554,4.250490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6860,2022,TOR,DPhelps,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,1.000000,0.333333,1.000000,6.112431,6.112431
6861,2022,MIA,EHernandez,7.907143,2.507143,3.153846,2.700000,0.198068,0.062802,0.135266,0.265957,1.350000,0.264706,0.668103,6.283860,5.016802
6862,2022,SDP,CStammen,9.000000,9.000000,1.000000,3.000000,0.166667,0.166667,0.000000,0.400000,3.000000,0.454545,0.526316,8.445764,6.086257
6863,2022,ATL,JStephens,0.000000,6.000000,0.000000,0.000000,0.000000,0.133333,-0.133333,0.384615,2.333333,0.384615,0.571429,5.112431,6.592800


In [ ]:
# save sp_adv_y as sp_adv_y_formatted as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/sp_adv_y_formatted.csv')
sp_adv_y_f.to_csv(filepath, index=False)

In [ ]:
# load sp_adv_y_f
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/sp_adv_y_formatted.csv')
sp_adv_y_f = pd.read_csv(filepath)

In [ ]:
sp_adv_y_f

,Season,Team,Starting Pitcher,sp/yr K/9,sp/yr BB/9,sp/yr K/BB,sp/yr HR/9,sp/yr K%,sp/yr BB%,sp/yr K-BB%,sp/yr AVG,sp/yr WHIP,sp/yr BABIP,sp/yr LOB%,sp/yr FIP,sp/yr xFIP
0,2002,COL,MHampton,3.727612,4.583955,0.813187,1.208955,0.088305,0.108592,-0.020286,0.312757,1.785448,0.317757,0.653215,5.525507,5.236269
1,2002,NYY,RClemens,9.600000,3.150000,3.047619,0.900000,0.250000,0.082031,0.167969,0.250000,1.305556,0.315574,0.682657,3.295408,3.317903
2,2002,CHC,JLieber,5.553191,0.765957,7.250000,0.957447,0.149485,0.020619,0.128866,0.276673,1.170213,0.295503,0.703448,3.387606,3.604062
3,2002,SEA,JMoyer,5.735549,1.950867,2.940000,1.092486,0.157895,0.053706,0.104189,0.230233,1.075145,0.243902,0.771350,4.032884,4.289525
4,2002,DET,JWeaver,5.547945,2.441096,2.272727,0.295890,0.147348,0.064833,0.082515,0.242950,1.191781,0.277635,0.698779,3.167554,4.250490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6860,2022,TOR,DPhelps,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,1.000000,0.333333,1.000000,6.112431,6.112431
6861,2022,MIA,EHernandez,7.907143,2.507143,3.153846,2.700000,0.198068,0.062802,0.135266,0.265957,1.350000,0.264706,0.668103,6.283860,5.016802
6862,2022,SDP,CStammen,9.000000,9.000000,1.000000,3.000000,0.166667,0.166667,0.000000,0.400000,3.000000,0.454545,0.526316,8.445764,6.086257
6863,2022,ATL,JStephens,0.000000,6.000000,0.000000,0.000000,0.000000,0.133333,-0.133333,0.384615,2.333333,0.384615,0.571429,5.112431,6.592800


In [ ]:
# format starting pitcher batted ball stats per year database
sp_bat_y_f = format_pitch_db(sp_bat_y, period='season', names=True)

In [ ]:
sp_bat_y_f

,Season,Team,Starting Pitcher,sp/yr GB/FB,sp/yr LD%,sp/yr GB%,sp/yr FB%,sp/yr IFFB%,sp/yr HR/FB,sp/yr IFH%,sp/yr BUH%,sp/yr Soft%,sp/yr Med%,sp/yr Hard%
0,2002,CHC,JLieber,1.216049,0.222944,0.426407,0.350649,0.104938,0.092593,0.055838,0.150000,0.192946,0.603734,0.203319
1,2002,COL,MHampton,1.684492,0.233588,0.480916,0.285496,0.058824,0.128342,0.082540,0.454545,0.151652,0.621622,0.226727
2,2002,NYY,RClemens,1.274854,0.214141,0.440404,0.345455,0.093567,0.105263,0.059633,0.272727,0.156436,0.649505,0.194059
3,2002,ANA,KAppier,0.764925,0.185886,0.352840,0.461274,0.111940,0.085821,0.058537,0.636364,0.155405,0.625000,0.219595
4,2002,SEA,JMoyer,0.871711,0.199719,0.372714,0.427567,0.154605,0.092105,0.045283,0.285714,0.186207,0.615172,0.198621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6860,2022,COL,AGomber,1.096154,0.215827,0.410072,0.374101,0.048077,0.125000,0.052632,0.500000,0.152482,0.524823,0.322695
6861,2022,HOU,FValdez,4.159091,0.174545,0.665455,0.160000,0.090909,0.125000,0.057377,0.400000,0.162162,0.536937,0.300901
6862,2022,SDP,CStammen,1.000000,0.333333,0.333333,0.333333,0.000000,0.250000,0.000000,0.000000,0.083333,0.750000,0.166667
6863,2022,ATL,JStephens,1.000000,0.538462,0.230769,0.230769,0.000000,0.000000,0.000000,0.000000,0.230769,0.538462,0.230769


In [ ]:
# save sp_bat_y_f as sp_bat_y_formatted as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/sp_bat_y_formatted.csv')
sp_bat_y_f.to_csv(filepath, index=False)

In [ ]:
# load sp_bat_y_f
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/sp_bat_y_formatted.csv')
sp_bat_y_f = pd.read_csv(filepath)

In [ ]:
sp_bat_y_f

,Season,Team,Starting Pitcher,sp/yr GB/FB,sp/yr LD%,sp/yr GB%,sp/yr FB%,sp/yr IFFB%,sp/yr HR/FB,sp/yr IFH%,sp/yr BUH%,sp/yr Soft%,sp/yr Med%,sp/yr Hard%
0,2002,CHC,JLieber,1.216049,0.222944,0.426407,0.350649,0.104938,0.092593,0.055838,0.150000,0.192946,0.603734,0.203319
1,2002,COL,MHampton,1.684492,0.233588,0.480916,0.285496,0.058824,0.128342,0.082540,0.454545,0.151652,0.621622,0.226727
2,2002,NYY,RClemens,1.274854,0.214141,0.440404,0.345455,0.093567,0.105263,0.059633,0.272727,0.156436,0.649505,0.194059
3,2002,ANA,KAppier,0.764925,0.185886,0.352840,0.461274,0.111940,0.085821,0.058537,0.636364,0.155405,0.625000,0.219595
4,2002,SEA,JMoyer,0.871711,0.199719,0.372714,0.427567,0.154605,0.092105,0.045283,0.285714,0.186207,0.615172,0.198621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6860,2022,COL,AGomber,1.096154,0.215827,0.410072,0.374101,0.048077,0.125000,0.052632,0.500000,0.152482,0.524823,0.322695
6861,2022,HOU,FValdez,4.159091,0.174545,0.665455,0.160000,0.090909,0.125000,0.057377,0.400000,0.162162,0.536937,0.300901
6862,2022,SDP,CStammen,1.000000,0.333333,0.333333,0.333333,0.000000,0.250000,0.000000,0.000000,0.083333,0.750000,0.166667
6863,2022,ATL,JStephens,1.000000,0.538462,0.230769,0.230769,0.000000,0.000000,0.000000,0.000000,0.230769,0.538462,0.230769


In [ ]:
# format starting pitcher advanced stats per game database
sp_adv_g_f = format_pitch_db(sp_adv_g, period='game', names=True)

In [ ]:
sp_adv_g_f

,Date,Team,Starting Pitcher,sp/g K/9,sp/g BB/9,sp/g K/BB,sp/g HR/9,sp/g K%,sp/g BB%,sp/g K-BB%,sp/g AVG,sp/g WHIP,sp/g BABIP,sp/g LOB%,sp/g FIP,sp/g xFIP
0,2002-03-31,ANA,JWashburn,10.800000,1.800000,6.00,1.800000,0.260870,0.043478,0.217391,0.363636,1.800000,0.466667,0.526316,3.762074,3.946275
1,2002-03-31,CLE,BColon,5.000000,2.000000,2.50,0.000000,0.151515,0.060606,0.090909,0.161290,0.777778,0.192308,1.000000,2.517630,3.755053
2,2002-04-01,ATL,TGlavine,9.947368,2.842105,3.50,1.421053,0.269231,0.076923,0.192308,0.208333,1.105263,0.250000,0.892857,3.751548,4.336581
3,2002-04-01,PHI,RPerson,5.400000,7.200000,0.75,3.600000,0.130435,0.173913,-0.043478,0.157895,1.400000,0.071429,0.238095,9.362074,6.389435
4,2002-04-01,BOS,PMartinez,12.000000,6.000000,2.00,0.000000,0.173913,0.086957,0.086957,0.500000,3.666667,0.600000,0.384615,4.295408,6.615575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98959,2022-10-05,SFG,JBrebbia,9.000000,0.000000,1.00,0.000000,0.200000,0.000000,0.200000,0.400000,2.000000,0.500000,0.500000,1.112431,2.592800
98960,2022-10-05,LAA,SOhtani,10.800000,1.800000,6.00,0.000000,0.352941,0.058824,0.294118,0.066667,0.400000,0.100000,0.500000,1.312431,2.496727
98961,2022-10-05,OAK,KWaldichuk,5.142857,1.285714,4.00,0.000000,0.160000,0.040000,0.120000,0.130435,0.571429,0.157895,1.000000,2.826717,4.307086
98962,2022-10-05,SEA,MGonzales,6.428571,2.571429,2.50,0.000000,0.161290,0.064516,0.096774,0.275862,1.428571,0.333333,0.600000,2.541002,5.078779


In [ ]:
# save sp_adv_g_f as sp_bat_y_formatted as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/sp_adv_g_formatted.csv')
sp_adv_g_f.to_csv(filepath, index=False)

In [ ]:
# load sp_adv_g_f database
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/sp_adv_g_formatted.csv')
sp_adv_g_f = pd.read_csv(filepath)

In [ ]:
sp_adv_g_f

,Date,Team,Starting Pitcher,sp/g K/9,sp/g BB/9,sp/g K/BB,sp/g HR/9,sp/g K%,sp/g BB%,sp/g K-BB%,sp/g AVG,sp/g WHIP,sp/g BABIP,sp/g LOB%,sp/g FIP,sp/g xFIP
0,2002-03-31,ANA,JWashburn,10.800000,1.800000,6.00,1.800000,0.260870,0.043478,0.217391,0.363636,1.800000,0.466667,0.526316,3.762074,3.946275
1,2002-03-31,CLE,BColon,5.000000,2.000000,2.50,0.000000,0.151515,0.060606,0.090909,0.161290,0.777778,0.192308,1.000000,2.517630,3.755053
2,2002-04-01,ATL,TGlavine,9.947368,2.842105,3.50,1.421053,0.269231,0.076923,0.192308,0.208333,1.105263,0.250000,0.892857,3.751548,4.336581
3,2002-04-01,PHI,RPerson,5.400000,7.200000,0.75,3.600000,0.130435,0.173913,-0.043478,0.157895,1.400000,0.071429,0.238095,9.362074,6.389435
4,2002-04-01,BOS,PMartinez,12.000000,6.000000,2.00,0.000000,0.173913,0.086957,0.086957,0.500000,3.666667,0.600000,0.384615,4.295408,6.615575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98959,2022-10-05,SFG,JBrebbia,9.000000,0.000000,1.00,0.000000,0.200000,0.000000,0.200000,0.400000,2.000000,0.500000,0.500000,1.112431,2.592800
98960,2022-10-05,LAA,SOhtani,10.800000,1.800000,6.00,0.000000,0.352941,0.058824,0.294118,0.066667,0.400000,0.100000,0.500000,1.312431,2.496727
98961,2022-10-05,OAK,KWaldichuk,5.142857,1.285714,4.00,0.000000,0.160000,0.040000,0.120000,0.130435,0.571429,0.157895,1.000000,2.826717,4.307086
98962,2022-10-05,SEA,MGonzales,6.428571,2.571429,2.50,0.000000,0.161290,0.064516,0.096774,0.275862,1.428571,0.333333,0.600000,2.541002,5.078779


In [ ]:
# format starting pitcher batted stats per game database
sp_bat_g_f = format_pitch_db(sp_bat_g, period='game', names=True)

In [ ]:
sp_bat_g_f

,Date,Team,Starting Pitcher,sp/g GB/FB,sp/g LD%,sp/g GB%,sp/g FB%,sp/g IFFB%,sp/g HR/FB,sp/g IFH%,sp/g BUH%,sp/g Soft%,sp/g Med%,sp/g Hard%
0,2002-03-31,ANA,JWashburn,0.300000,0.187500,0.187500,0.625000,0.100000,0.100000,0.000000,0.0,0.250000,0.375000,0.375000
1,2002-03-31,CLE,BColon,1.625000,0.192308,0.500000,0.307692,0.000000,0.000000,0.076923,0.0,0.307692,0.538462,0.153846
2,2002-04-01,ATL,TGlavine,0.250000,0.117647,0.176471,0.705882,0.166667,0.083333,0.000000,0.0,0.235294,0.352941,0.411765
3,2002-04-01,PHI,RPerson,0.875000,0.062500,0.437500,0.500000,0.000000,0.250000,0.142857,0.0,0.375000,0.375000,0.250000
4,2002-04-01,BOS,PMartinez,0.600000,0.428571,0.214286,0.357143,0.000000,0.000000,0.000000,1.0,0.066667,0.600000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98959,2022-10-05,SFG,JBrebbia,3.000000,0.000000,0.750000,0.250000,0.000000,0.000000,0.000000,0.0,0.250000,0.500000,0.250000
98960,2022-10-05,LAA,SOhtani,1.250000,0.100000,0.500000,0.400000,0.250000,0.000000,0.000000,0.0,0.100000,0.600000,0.300000
98961,2022-10-05,OAK,KWaldichuk,1.142857,0.210526,0.421053,0.368421,0.142857,0.000000,0.000000,0.0,0.210526,0.631579,0.157895
98962,2022-10-05,SEA,MGonzales,0.666667,0.166667,0.333333,0.500000,0.083333,0.000000,0.250000,0.0,0.416667,0.416667,0.166667


In [ ]:
# save sp_bat_g_f as sp_bat_y_formatted as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/sp_bat_g_formatted.csv')
sp_bat_g_f.to_csv(filepath, index=False)

In [ ]:
# load sp_bat_g_f
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/sp_bat_g_formatted.csv')
sp_bat_g_f = pd.read_csv(filepath)

In [ ]:
sp_bat_g_f

,Date,Team,Starting Pitcher,sp/g GB/FB,sp/g LD%,sp/g GB%,sp/g FB%,sp/g IFFB%,sp/g HR/FB,sp/g IFH%,sp/g BUH%,sp/g Soft%,sp/g Med%,sp/g Hard%
0,2002-03-31,ANA,JWashburn,0.300000,0.187500,0.187500,0.625000,0.100000,0.100000,0.000000,0.0,0.250000,0.375000,0.375000
1,2002-03-31,CLE,BColon,1.625000,0.192308,0.500000,0.307692,0.000000,0.000000,0.076923,0.0,0.307692,0.538462,0.153846
2,2002-04-01,ATL,TGlavine,0.250000,0.117647,0.176471,0.705882,0.166667,0.083333,0.000000,0.0,0.235294,0.352941,0.411765
3,2002-04-01,PHI,RPerson,0.875000,0.062500,0.437500,0.500000,0.000000,0.250000,0.142857,0.0,0.375000,0.375000,0.250000
4,2002-04-01,BOS,PMartinez,0.600000,0.428571,0.214286,0.357143,0.000000,0.000000,0.000000,1.0,0.066667,0.600000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98959,2022-10-05,SFG,JBrebbia,3.000000,0.000000,0.750000,0.250000,0.000000,0.000000,0.000000,0.0,0.250000,0.500000,0.250000
98960,2022-10-05,LAA,SOhtani,1.250000,0.100000,0.500000,0.400000,0.250000,0.000000,0.000000,0.0,0.100000,0.600000,0.300000
98961,2022-10-05,OAK,KWaldichuk,1.142857,0.210526,0.421053,0.368421,0.142857,0.000000,0.000000,0.0,0.210526,0.631579,0.157895
98962,2022-10-05,SEA,MGonzales,0.666667,0.166667,0.333333,0.500000,0.083333,0.000000,0.250000,0.0,0.416667,0.416667,0.166667


In [ ]:
# format relief pitcher advanced stats per season database
rp_adv_y_f = format_pitch_db(rp_adv_y, period='season')

In [ ]:
rp_adv_y_f

,Season,Team,rp/yr K/9,rp/yr BB/9,rp/yr K/BB,rp/yr HR/9,rp/yr K%,rp/yr BB%,rp/yr K-BB%,rp/yr AVG,rp/yr WHIP,rp/yr BABIP,rp/yr LOB%,rp/yr FIP,rp/yr xFIP
0,2002,LAD,7.441696,3.320141,2.241379,1.011307,0.194903,0.086957,0.107946,0.237374,1.265724,0.270666,0.763246,3.983983,3.995979
1,2002,STL,6.533081,3.402646,1.920000,0.816635,0.175503,0.091408,0.084095,0.233091,1.224953,0.259740,0.772453,3.914815,4.122069
2,2002,CIN,7.087108,3.418118,2.073395,0.815331,0.186161,0.089786,0.096376,0.247427,1.301394,0.283591,0.727585,3.829670,4.097426
3,2002,BOS,7.251656,3.238411,2.239264,0.973510,0.187854,0.083891,0.103963,0.250731,1.306843,0.285071,0.709921,4.054790,4.277335
4,2002,MIL,7.034960,4.220976,1.666667,1.086412,0.181776,0.109066,0.072711,0.237480,1.341689,0.260810,0.750487,4.511547,4.531814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2022,SFG,8.058462,3.336923,2.414938,0.900000,0.207265,0.085826,0.121439,0.259436,1.375385,0.309637,0.705097,3.830893,4.047702
626,2022,CIN,9.283542,4.341119,2.138514,1.202607,0.234618,0.109711,0.124907,0.247746,1.422596,0.300912,0.708573,4.438884,4.354234
627,2022,LAA,8.402364,3.433877,2.446903,1.230726,0.222535,0.090946,0.131590,0.226697,1.227349,0.262993,0.701372,4.309392,4.186158
628,2022,STL,7.986581,3.386814,2.358140,0.929405,0.212134,0.089958,0.122176,0.233803,1.247958,0.276448,0.752226,3.919315,4.001935


In [ ]:
# save rp_adv_y_f as rp_adv_y_formatted as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/rp_adv_y_formatted.csv')
rp_adv_y_f.to_csv(filepath, index=False)

In [ ]:
# load rp_adv_y_f
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/rp_adv_y_formatted.csv')
rp_adv_y_f = pd.read_csv(filepath)

In [ ]:
rp_adv_y_f

,Season,Team,rp/yr K/9,rp/yr BB/9,rp/yr K/BB,rp/yr HR/9,rp/yr K%,rp/yr BB%,rp/yr K-BB%,rp/yr AVG,rp/yr WHIP,rp/yr BABIP,rp/yr LOB%,rp/yr FIP,rp/yr xFIP
0,2002,LAD,7.441696,3.320141,2.241379,1.011307,0.194903,0.086957,0.107946,0.237374,1.265724,0.270666,0.763246,3.983983,3.995979
1,2002,STL,6.533081,3.402646,1.920000,0.816635,0.175503,0.091408,0.084095,0.233091,1.224953,0.259740,0.772453,3.914815,4.122069
2,2002,CIN,7.087108,3.418118,2.073395,0.815331,0.186161,0.089786,0.096376,0.247427,1.301394,0.283591,0.727585,3.829670,4.097426
3,2002,BOS,7.251656,3.238411,2.239264,0.973510,0.187854,0.083891,0.103963,0.250731,1.306843,0.285071,0.709921,4.054790,4.277335
4,2002,MIL,7.034960,4.220976,1.666667,1.086412,0.181776,0.109066,0.072711,0.237480,1.341689,0.260810,0.750487,4.511547,4.531814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2022,SFG,8.058462,3.336923,2.414938,0.900000,0.207265,0.085826,0.121439,0.259436,1.375385,0.309637,0.705097,3.830893,4.047702
626,2022,CIN,9.283542,4.341119,2.138514,1.202607,0.234618,0.109711,0.124907,0.247746,1.422596,0.300912,0.708573,4.438884,4.354234
627,2022,LAA,8.402364,3.433877,2.446903,1.230726,0.222535,0.090946,0.131590,0.226697,1.227349,0.262993,0.701372,4.309392,4.186158
628,2022,STL,7.986581,3.386814,2.358140,0.929405,0.212134,0.089958,0.122176,0.233803,1.247958,0.276448,0.752226,3.919315,4.001935


In [ ]:
# format relief pitcher batted ball stats per season database
rp_bat_y_f = format_pitch_db(rp_bat_y, period='season')

In [ ]:
rp_bat_y_f

,Season,Team,rp/yr GB/FB,rp/yr LD%,rp/yr GB%,rp/yr FB%,rp/yr IFFB%,rp/yr HR/FB,rp/yr IFH%,rp/yr BUH%,rp/yr Soft%,rp/yr Med%,rp/yr Hard%
0,2002,LAD,1.226453,0.200719,0.440288,0.358993,0.128257,0.106212,0.052288,0.233333,0.177590,0.620860,0.201550
1,2002,STL,1.303605,0.215255,0.444085,0.340659,0.121442,0.091082,0.064047,0.075000,0.160151,0.622951,0.216898
2,2002,CIN,1.214765,0.212880,0.431723,0.355397,0.127517,0.087248,0.074586,0.232143,0.167436,0.606813,0.225751
3,2002,BOS,1.052830,0.199411,0.410596,0.389993,0.126415,0.092453,0.051971,0.181818,0.145547,0.647357,0.207096
4,2002,MIL,1.003466,0.208761,0.396304,0.394935,0.145581,0.105719,0.058722,0.125000,0.170297,0.617822,0.211881
...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2022,SFG,1.339339,0.196493,0.460031,0.343476,0.091592,0.097598,0.079596,0.260870,0.185015,0.536188,0.278797
626,2022,CIN,1.011679,0.194623,0.405026,0.400351,0.121168,0.119708,0.069264,0.250000,0.169079,0.536190,0.294731
627,2022,LAA,1.001511,0.200362,0.400121,0.399517,0.089124,0.122357,0.064857,0.363636,0.151876,0.541989,0.306135
628,2022,STL,1.403636,0.189951,0.473039,0.337010,0.094545,0.107273,0.053109,0.142857,0.176792,0.538275,0.284933


In [ ]:
# save rp_bat_y as rp_bat_y_formatted as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/rp_bat_y_formatted.csv')
rp_bat_y_f.to_csv(filepath, index=False)

In [ ]:
# load rp_bat_y_f
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/rp_bat_y_formatted.csv')
rp_bat_y_f = pd.read_csv(filepath)

In [ ]:
rp_bat_y_f

,Season,Team,rp/yr GB/FB,rp/yr LD%,rp/yr GB%,rp/yr FB%,rp/yr IFFB%,rp/yr HR/FB,rp/yr IFH%,rp/yr BUH%,rp/yr Soft%,rp/yr Med%,rp/yr Hard%
0,2002,LAD,1.226453,0.200719,0.440288,0.358993,0.128257,0.106212,0.052288,0.233333,0.177590,0.620860,0.201550
1,2002,STL,1.303605,0.215255,0.444085,0.340659,0.121442,0.091082,0.064047,0.075000,0.160151,0.622951,0.216898
2,2002,CIN,1.214765,0.212880,0.431723,0.355397,0.127517,0.087248,0.074586,0.232143,0.167436,0.606813,0.225751
3,2002,BOS,1.052830,0.199411,0.410596,0.389993,0.126415,0.092453,0.051971,0.181818,0.145547,0.647357,0.207096
4,2002,MIL,1.003466,0.208761,0.396304,0.394935,0.145581,0.105719,0.058722,0.125000,0.170297,0.617822,0.211881
...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2022,SFG,1.339339,0.196493,0.460031,0.343476,0.091592,0.097598,0.079596,0.260870,0.185015,0.536188,0.278797
626,2022,CIN,1.011679,0.194623,0.405026,0.400351,0.121168,0.119708,0.069264,0.250000,0.169079,0.536190,0.294731
627,2022,LAA,1.001511,0.200362,0.400121,0.399517,0.089124,0.122357,0.064857,0.363636,0.151876,0.541989,0.306135
628,2022,STL,1.403636,0.189951,0.473039,0.337010,0.094545,0.107273,0.053109,0.142857,0.176792,0.538275,0.284933


In [ ]:
# format relief pitcher adv stats per game database
rp_adv_g_f = format_pitch_db(rp_adv_g, period='game')

In [ ]:
rp_adv_g_f

,Date,Team,rp/g K/9,rp/g BB/9,rp/g K/BB,rp/g HR/9,rp/g K%,rp/g BB%,rp/g K-BB%,rp/g AVG,rp/g WHIP,rp/g BABIP,rp/g LOB%,rp/g FIP,rp/g xFIP
0,2002-03-31,ANA,6.750,4.500,1.500000,0.0,0.200000,0.133333,0.066667,0.230769,1.250000,0.300000,0.800000,2.962074,5.050225
1,2002-04-01,ATL,10.125,0.000,3.000000,0.0,0.375000,0.000000,0.375000,0.000000,0.000000,0.000000,1.000000,0.712074,2.278187
2,2002-04-01,PHI,3.000,0.000,1.000000,0.0,0.066667,0.000000,0.066667,0.333333,1.666667,0.357143,0.800000,2.295408,4.151542
3,2002-04-01,BOS,4.500,6.000,0.750000,0.0,0.103448,0.137931,-0.034483,0.250000,1.500000,0.250000,0.636364,4.962074,6.586192
4,2002-04-01,TOR,6.750,13.500,0.500000,0.0,0.138889,0.277778,-0.138889,0.280000,2.550000,0.333333,0.705882,5.962074,8.050225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96506,2022-10-05,SFG,12.375,1.125,11.000000,0.0,0.423077,0.038462,0.384615,0.080000,0.375000,0.142857,1.000000,0.737431,1.847708
96507,2022-10-05,LAA,6.000,3.000,2.000000,3.0,0.142857,0.071429,0.071429,0.250000,1.333333,0.200000,0.769231,7.112431,5.739836
96508,2022-10-05,OAK,0.000,0.000,0.000000,4.5,0.000000,0.000000,0.000000,0.375000,1.500000,0.285714,0.625000,9.612431,4.592800
96509,2022-10-05,DET,3.375,10.125,0.333333,0.0,0.076923,0.230769,-0.153846,0.200000,1.875000,0.222222,0.800000,5.737431,7.402847


In [ ]:
# save rp_adv_g_f as rp_bat_y_formatted as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/rp_adv_g_formatted.csv')
rp_adv_g_f.to_csv(filepath, index=False)

In [ ]:
# load rp_adv_g_f
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/rp_adv_g_formatted.csv')
rp_adv_g_f = pd.read_csv(filepath)

In [ ]:
rp_adv_g_f

,Date,Team,rp/g K/9,rp/g BB/9,rp/g K/BB,rp/g HR/9,rp/g K%,rp/g BB%,rp/g K-BB%,rp/g AVG,rp/g WHIP,rp/g BABIP,rp/g LOB%,rp/g FIP,rp/g xFIP
0,2002-03-31,ANA,6.750,4.500,1.500000,0.0,0.200000,0.133333,0.066667,0.230769,1.250000,0.300000,0.800000,2.962074,5.050225
1,2002-04-01,ATL,10.125,0.000,3.000000,0.0,0.375000,0.000000,0.375000,0.000000,0.000000,0.000000,1.000000,0.712074,2.278187
2,2002-04-01,PHI,3.000,0.000,1.000000,0.0,0.066667,0.000000,0.066667,0.333333,1.666667,0.357143,0.800000,2.295408,4.151542
3,2002-04-01,BOS,4.500,6.000,0.750000,0.0,0.103448,0.137931,-0.034483,0.250000,1.500000,0.250000,0.636364,4.962074,6.586192
4,2002-04-01,TOR,6.750,13.500,0.500000,0.0,0.138889,0.277778,-0.138889,0.280000,2.550000,0.333333,0.705882,5.962074,8.050225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96506,2022-10-05,SFG,12.375,1.125,11.000000,0.0,0.423077,0.038462,0.384615,0.080000,0.375000,0.142857,1.000000,0.737431,1.847708
96507,2022-10-05,LAA,6.000,3.000,2.000000,3.0,0.142857,0.071429,0.071429,0.250000,1.333333,0.200000,0.769231,7.112431,5.739836
96508,2022-10-05,OAK,0.000,0.000,0.000000,4.5,0.000000,0.000000,0.000000,0.375000,1.500000,0.285714,0.625000,9.612431,4.592800
96509,2022-10-05,DET,3.375,10.125,0.333333,0.0,0.076923,0.230769,-0.153846,0.200000,1.875000,0.222222,0.800000,5.737431,7.402847


In [ ]:
# format relief pitcher batted ball stats per game database
rp_bat_g_f = format_pitch_db(rp_bat_g, period='game')

In [ ]:
rp_bat_g_f

,Date,Team,rp/g GB/FB,rp/g LD%,rp/g GB%,rp/g FB%,rp/g IFFB%,rp/g HR/FB,rp/g IFH%,rp/g BUH%,rp/g Soft%,rp/g Med%,rp/g Hard%
0,2002-03-31,ANA,0.666667,0.000000,0.400000,0.600000,0.166667,0.000000,0.000000,0.0,0.400000,0.400000,0.200000
1,2002-04-01,ATL,0.666667,0.000000,0.400000,0.600000,0.000000,0.000000,0.000000,0.0,0.400000,0.400000,0.200000
2,2002-04-01,PHI,1.750000,0.214286,0.500000,0.285714,0.250000,0.000000,0.142857,0.0,0.214286,0.357143,0.428571
3,2002-04-01,BOS,1.142857,0.210526,0.421053,0.368421,0.142857,0.000000,0.250000,0.0,0.150000,0.800000,0.050000
4,2002-04-01,TOR,0.500000,0.250000,0.250000,0.500000,0.300000,0.000000,0.000000,0.0,0.285714,0.476190,0.238095
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96506,2022-10-05,SFG,1.166667,0.071429,0.500000,0.428571,0.166667,0.000000,0.000000,0.0,0.214286,0.571429,0.214286
96507,2022-10-05,LAA,0.666667,0.090909,0.363636,0.545455,0.000000,0.166667,0.000000,0.0,0.090909,0.818182,0.090909
96508,2022-10-05,OAK,2.000000,0.250000,0.500000,0.250000,0.000000,0.500000,0.000000,0.0,0.000000,0.625000,0.375000
96509,2022-10-05,DET,2.000000,0.000000,0.666667,0.333333,0.000000,0.000000,0.166667,0.0,0.222222,0.555556,0.222222


In [ ]:
# save rp_bat_g_f as rp_bat_y_formatted as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/rp_bat_g_formatted.csv')
rp_bat_g_f.to_csv(filepath, index=False)

In [ ]:
# load rp_bat_g_f
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/rp_bat_g_formatted.csv')
rp_bat_g_f = pd.read_csv(filepath)

In [ ]:
rp_bat_g_f

,Date,Team,rp/g GB/FB,rp/g LD%,rp/g GB%,rp/g FB%,rp/g IFFB%,rp/g HR/FB,rp/g IFH%,rp/g BUH%,rp/g Soft%,rp/g Med%,rp/g Hard%
0,2002-03-31,ANA,0.666667,0.000000,0.400000,0.600000,0.166667,0.000000,0.000000,0.0,0.400000,0.400000,0.200000
1,2002-04-01,ATL,0.666667,0.000000,0.400000,0.600000,0.000000,0.000000,0.000000,0.0,0.400000,0.400000,0.200000
2,2002-04-01,PHI,1.750000,0.214286,0.500000,0.285714,0.250000,0.000000,0.142857,0.0,0.214286,0.357143,0.428571
3,2002-04-01,BOS,1.142857,0.210526,0.421053,0.368421,0.142857,0.000000,0.250000,0.0,0.150000,0.800000,0.050000
4,2002-04-01,TOR,0.500000,0.250000,0.250000,0.500000,0.300000,0.000000,0.000000,0.0,0.285714,0.476190,0.238095
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96506,2022-10-05,SFG,1.166667,0.071429,0.500000,0.428571,0.166667,0.000000,0.000000,0.0,0.214286,0.571429,0.214286
96507,2022-10-05,LAA,0.666667,0.090909,0.363636,0.545455,0.000000,0.166667,0.000000,0.0,0.090909,0.818182,0.090909
96508,2022-10-05,OAK,2.000000,0.250000,0.500000,0.250000,0.000000,0.500000,0.000000,0.0,0.000000,0.625000,0.375000
96509,2022-10-05,DET,2.000000,0.000000,0.666667,0.333333,0.000000,0.000000,0.166667,0.0,0.222222,0.555556,0.222222


In [ ]:
def format_batting(df, period,):
    # drop 'PA' column
    df = df.drop(columns=['PA'])
    # if pull% is a column in the database, drop pull%, cent%, and oppo% columns
    if 'Pull%' in df.columns:
        df = df.drop(columns=['Pull%','Cent%', 'Oppo%'])
    # change 'Tm' column to 'Team'
    df = df.rename(columns={'Tm':'Team'})
    # check period
    if period == 'season':
        df['Season'] = pd.to_datetime(df['Season'], format='%Y').dt.year
        exclude_cols = ['Season','Team']
        prefix = r'/yr '
    elif period == 'game':
        df['Date'] = pd.to_datetime(df['Date'])
        exclude_cols = ['Date','Team']
        prefix = r'/g '
    
    df = pd.concat([df[exclude_cols], df.drop(columns=exclude_cols).add_prefix(prefix)], axis=1)
    
    return df

In [ ]:
# format team adv batting per year
team_adv_bat_y_f = format_batting(team_adv_bat_y, period='season')

In [ ]:
team_adv_bat_y_f

,Season,Team,/yr BB%,/yr K%,/yr BB/K,/yr AVG,/yr OBP,/yr SLG,/yr OPS,/yr ISO,/yr BABIP,/yr wRC,/yr wRAA,/yr wOBA,/yr wRC+
0,2002,ANA,0.073020,0.127232,0.573913,0.282318,0.340714,0.432547,0.773260,0.150229,0.303239,810.797010,51.075588,0.335925,104.792946
1,2002,BAL,0.074147,0.162894,0.455186,0.246403,0.308620,0.403023,0.711643,0.156620,0.271109,654.365338,-77.618507,0.310501,89.893491
2,2002,BOS,0.086071,0.149084,0.577331,0.276596,0.345008,0.444149,0.789157,0.167553,0.302493,848.877642,88.555840,0.342839,107.079563
3,2002,CHW,0.089401,0.153351,0.582983,0.268084,0.337555,0.449473,0.787028,0.181389,0.286822,826.725055,81.292687,0.342026,105.439602
4,2002,CLE,0.088867,0.163961,0.542000,0.248755,0.321287,0.411580,0.732867,0.162825,0.270960,703.887840,-28.456233,0.320164,94.543848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2022,PHI,0.078657,0.224288,0.350697,0.253275,0.316639,0.422125,0.738764,0.168850,0.298842,752.397511,57.514122,0.321607,105.812719
626,2022,PIT,0.080514,0.253214,0.317969,0.222472,0.291193,0.363722,0.654915,0.141249,0.277238,577.138927,-98.877297,0.288710,84.226686
627,2022,STL,0.087105,0.198865,0.438010,0.252183,0.325268,0.420124,0.745392,0.167940,0.288732,784.408701,79.462824,0.325941,113.657725
628,2022,SDP,0.092955,0.214899,0.432555,0.240856,0.317894,0.381675,0.699569,0.140819,0.288547,698.402938,-7.686403,0.308077,101.810046


In [ ]:
# save team_adv_bat_y_f as team_adv_bat_y_formatted as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/team_adv_bat_y_formatted.csv')
team_adv_bat_y_f.to_csv(filepath, index=False)

In [ ]:
#load team_adv_bat_y_f
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/team_adv_bat_y_formatted.csv')
team_adv_bat_y_f = pd.read_csv(filepath)

In [ ]:
team_adv_bat_y_f

,Season,Team,/yr BB%,/yr K%,/yr BB/K,/yr AVG,/yr OBP,/yr SLG,/yr OPS,/yr ISO,/yr BABIP,/yr wRC,/yr wRAA,/yr wOBA,/yr wRC+
0,2002,ANA,0.073020,0.127232,0.573913,0.282318,0.340714,0.432547,0.773260,0.150229,0.303239,810.797010,51.075588,0.335925,104.792946
1,2002,BAL,0.074147,0.162894,0.455186,0.246403,0.308620,0.403023,0.711643,0.156620,0.271109,654.365338,-77.618507,0.310501,89.893491
2,2002,BOS,0.086071,0.149084,0.577331,0.276596,0.345008,0.444149,0.789157,0.167553,0.302493,848.877642,88.555840,0.342839,107.079563
3,2002,CHW,0.089401,0.153351,0.582983,0.268084,0.337555,0.449473,0.787028,0.181389,0.286822,826.725055,81.292687,0.342026,105.439602
4,2002,CLE,0.088867,0.163961,0.542000,0.248755,0.321287,0.411580,0.732867,0.162825,0.270960,703.887840,-28.456233,0.320164,94.543848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2022,PHI,0.078657,0.224288,0.350697,0.253275,0.316639,0.422125,0.738764,0.168850,0.298842,752.397511,57.514122,0.321607,105.812719
626,2022,PIT,0.080514,0.253214,0.317969,0.222472,0.291193,0.363722,0.654915,0.141249,0.277238,577.138927,-98.877297,0.288710,84.226686
627,2022,STL,0.087105,0.198865,0.438010,0.252183,0.325268,0.420124,0.745392,0.167940,0.288732,784.408701,79.462824,0.325941,113.657725
628,2022,SDP,0.092955,0.214899,0.432555,0.240856,0.317894,0.381675,0.699569,0.140819,0.288547,698.402938,-7.686403,0.308077,101.810046


In [ ]:
# format team bat batting per year
team_bat_bat_y_f = format_batting(team_bat_bat_y, period='season')

In [ ]:
team_bat_bat_y_f

,Season,Team,/yr GB/FB,/yr LD%,/yr GB%,/yr FB%,/yr IFFB%,/yr HR/FB,/yr IFH%,/yr BUH%,/yr Soft%,/yr Med%,/yr Hard%
0,2002,ANA,1.036041,0.224862,0.394430,0.380709,0.109736,0.081764,0.059709,0.271845,0.146155,0.640835,0.213009
1,2002,BAL,1.045429,0.197951,0.409931,0.392118,0.147076,0.093697,0.058664,0.177083,0.162852,0.610421,0.226728
2,2002,BOS,1.216507,0.214997,0.430841,0.354162,0.118421,0.105861,0.067847,0.320000,0.140730,0.622483,0.236787
3,2002,CHW,1.022497,0.205830,0.401502,0.392668,0.123735,0.122047,0.051155,0.338843,0.145932,0.619673,0.234395
4,2002,CLE,1.243020,0.199321,0.443715,0.356965,0.125000,0.121827,0.055130,0.220930,0.156841,0.623804,0.219355
...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2022,PHI,1.167213,0.203230,0.429122,0.367647,0.112787,0.134426,0.075281,0.542857,0.169974,0.526177,0.303849
626,2022,PIT,1.219020,0.196452,0.441430,0.362118,0.100865,0.113833,0.063239,0.274510,0.169928,0.534758,0.295314
627,2022,STL,1.046690,0.195026,0.411669,0.393305,0.124113,0.116430,0.062676,0.444444,0.182407,0.517130,0.300463
628,2022,SDP,1.130185,0.197210,0.425926,0.376864,0.102744,0.097639,0.064935,0.369565,0.181732,0.522598,0.295671


In [ ]:
# save team_bat_bat_y_f as team_adv_bat_y_formatted as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/team_bat_bat_y_formatted.csv')
team_bat_bat_y_f.to_csv(filepath, index=False)

In [ ]:
# load team_bat_bat_y_f
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/team_bat_bat_y_formatted.csv')
team_bat_bat_y_f = pd.read_csv(filepath)

In [ ]:
team_bat_bat_y_f

,Season,Team,/yr GB/FB,/yr LD%,/yr GB%,/yr FB%,/yr IFFB%,/yr HR/FB,/yr IFH%,/yr BUH%,/yr Soft%,/yr Med%,/yr Hard%
0,2002,ANA,1.036041,0.224862,0.394430,0.380709,0.109736,0.081764,0.059709,0.271845,0.146155,0.640835,0.213009
1,2002,BAL,1.045429,0.197951,0.409931,0.392118,0.147076,0.093697,0.058664,0.177083,0.162852,0.610421,0.226728
2,2002,BOS,1.216507,0.214997,0.430841,0.354162,0.118421,0.105861,0.067847,0.320000,0.140730,0.622483,0.236787
3,2002,CHW,1.022497,0.205830,0.401502,0.392668,0.123735,0.122047,0.051155,0.338843,0.145932,0.619673,0.234395
4,2002,CLE,1.243020,0.199321,0.443715,0.356965,0.125000,0.121827,0.055130,0.220930,0.156841,0.623804,0.219355
...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2022,PHI,1.167213,0.203230,0.429122,0.367647,0.112787,0.134426,0.075281,0.542857,0.169974,0.526177,0.303849
626,2022,PIT,1.219020,0.196452,0.441430,0.362118,0.100865,0.113833,0.063239,0.274510,0.169928,0.534758,0.295314
627,2022,STL,1.046690,0.195026,0.411669,0.393305,0.124113,0.116430,0.062676,0.444444,0.182407,0.517130,0.300463
628,2022,SDP,1.130185,0.197210,0.425926,0.376864,0.102744,0.097639,0.064935,0.369565,0.181732,0.522598,0.295671


In [ ]:
# format team adv batting per game
team_adv_bat_g_f = format_batting(team_adv_bat_g, period='game')

In [ ]:
team_adv_bat_g_f

,Date,Team,/g BB%,/g K%,/g BB/K,/g AVG,/g OBP,/g SLG,/g OPS,/g ISO,/g BABIP,/g wRC,/g wRAA,/g wOBA,/g wRC+
0,2002-03-31,ANA,0.060606,0.151515,0.400000,0.161290,0.212121,0.193548,0.405670,0.032258,0.192308,0.226965,-3.735546,0.188861,14.923632
1,2002-03-31,CLE,0.078947,0.236842,0.333333,0.314286,0.368421,0.428571,0.796992,0.114286,0.400000,5.416478,0.853586,0.353108,116.526569
2,2002-04-01,ATL,0.105263,0.105263,1.000000,0.235294,0.315789,0.500000,0.815789,0.264706,0.214286,5.320222,0.757331,0.350041,118.903758
3,2002-04-01,PHI,0.058824,0.294118,0.200000,0.161290,0.205882,0.322581,0.528463,0.161290,0.190476,1.310226,-2.772361,0.227190,36.342876
4,2002-04-01,BOS,0.250000,0.153846,1.625000,0.342105,0.500000,0.684211,1.184211,0.342105,0.333333,13.417794,7.173837,0.492940,207.710284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98969,2022-10-05,SFG,0.090909,0.181818,0.500000,0.324324,0.386364,0.594595,0.980958,0.270270,0.321429,8.522521,3.491277,0.409624,169.094419
98970,2022-10-05,LAA,0.030303,0.121212,0.250000,0.193548,0.242424,0.387097,0.629521,0.193548,0.192308,2.799228,-0.974205,0.272586,82.136999
98971,2022-10-05,OAK,0.064516,0.258065,0.250000,0.148148,0.193548,0.296296,0.489845,0.148148,0.150000,1.063355,-2.481385,0.208989,41.431650
98972,2022-10-05,DET,0.054054,0.270270,0.200000,0.228571,0.270270,0.257143,0.527413,0.028571,0.320000,2.137736,-2.093082,0.238538,56.439381


In [ ]:
# save team_adv_bat_g_f as team_adv_bat_y_formatted as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/team_adv_bat_g_formatted.csv')
team_adv_bat_g_f.to_csv(filepath, index=False)

In [ ]:
# load team_adv_bat_g_f
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/team_adv_bat_g_formatted.csv')
team_adv_bat_g_f = pd.read_csv(filepath)

In [ ]:
team_adv_bat_g_f

,Date,Team,/g BB%,/g K%,/g BB/K,/g AVG,/g OBP,/g SLG,/g OPS,/g ISO,/g BABIP,/g wRC,/g wRAA,/g wOBA,/g wRC+
0,2002-03-31,ANA,0.060606,0.151515,0.400000,0.161290,0.212121,0.193548,0.405670,0.032258,0.192308,0.226965,-3.735546,0.188861,14.923632
1,2002-03-31,CLE,0.078947,0.236842,0.333333,0.314286,0.368421,0.428571,0.796992,0.114286,0.400000,5.416478,0.853586,0.353108,116.526569
2,2002-04-01,ATL,0.105263,0.105263,1.000000,0.235294,0.315789,0.500000,0.815789,0.264706,0.214286,5.320222,0.757331,0.350041,118.903758
3,2002-04-01,PHI,0.058824,0.294118,0.200000,0.161290,0.205882,0.322581,0.528463,0.161290,0.190476,1.310226,-2.772361,0.227190,36.342876
4,2002-04-01,BOS,0.250000,0.153846,1.625000,0.342105,0.500000,0.684211,1.184211,0.342105,0.333333,13.417794,7.173837,0.492940,207.710284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98969,2022-10-05,SFG,0.090909,0.181818,0.500000,0.324324,0.386364,0.594595,0.980958,0.270270,0.321429,8.522521,3.491277,0.409624,169.094419
98970,2022-10-05,LAA,0.030303,0.121212,0.250000,0.193548,0.242424,0.387097,0.629521,0.193548,0.192308,2.799228,-0.974205,0.272586,82.136999
98971,2022-10-05,OAK,0.064516,0.258065,0.250000,0.148148,0.193548,0.296296,0.489845,0.148148,0.150000,1.063355,-2.481385,0.208989,41.431650
98972,2022-10-05,DET,0.054054,0.270270,0.200000,0.228571,0.270270,0.257143,0.527413,0.028571,0.320000,2.137736,-2.093082,0.238538,56.439381


In [ ]:
# format team bat batting per game
team_bat_bat_g_f = format_batting(team_bat_bat_g, period='game')

In [ ]:
team_bat_bat_g_f

,Date,Team,/g GB/FB,/g LD%,/g GB%,/g FB%,/g IFFB%,/g HR/FB,/g IFH%,/g BUH%,/g Soft%,/g Med%,/g Hard%
0,2002-03-31,ANA,1.625000,0.192308,0.500000,0.307692,0.000000,0.000000,0.076923,0.0,0.307692,0.538462,0.153846
1,2002-03-31,CLE,0.437500,0.115385,0.269231,0.615385,0.125000,0.062500,0.000000,0.0,0.307692,0.384615,0.307692
2,2002-04-01,ATL,1.166667,0.133333,0.466667,0.400000,0.083333,0.166667,0.142857,0.0,0.300000,0.366667,0.333333
3,2002-04-01,PHI,0.333333,0.090909,0.227273,0.681818,0.133333,0.066667,0.000000,0.0,0.272727,0.363636,0.363636
4,2002-04-01,BOS,0.562500,0.166667,0.300000,0.533333,0.187500,0.250000,0.000000,0.0,0.225806,0.419355,0.354839
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98969,2022-10-05,SFG,1.875000,0.258065,0.483871,0.258065,0.000000,0.375000,0.000000,0.0,0.161290,0.580645,0.258065
98970,2022-10-05,LAA,1.333333,0.222222,0.444444,0.333333,0.111111,0.111111,0.000000,0.0,0.148148,0.629630,0.222222
98971,2022-10-05,OAK,0.900000,0.095238,0.428571,0.476190,0.100000,0.100000,0.000000,0.0,0.095238,0.714286,0.190476
98972,2022-10-05,DET,0.615385,0.160000,0.320000,0.520000,0.076923,0.000000,0.250000,0.0,0.440000,0.400000,0.160000


In [ ]:
# save team_bat_bat_g_f as team_adv_bat_y_formatted as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/team_bat_bat_g_formatted.csv')
team_bat_bat_g_f.to_csv(filepath, index=False)

In [ ]:
# load team_bat_bat_g_f
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/team_bat_bat_g_formatted.csv')
team_bat_bat_g_f = pd.read_csv(filepath)

In [ ]:
team_bat_bat_g_f

,Date,Team,/g GB/FB,/g LD%,/g GB%,/g FB%,/g IFFB%,/g HR/FB,/g IFH%,/g BUH%,/g Soft%,/g Med%,/g Hard%
0,2002-03-31,ANA,1.625000,0.192308,0.500000,0.307692,0.000000,0.000000,0.076923,0.0,0.307692,0.538462,0.153846
1,2002-03-31,CLE,0.437500,0.115385,0.269231,0.615385,0.125000,0.062500,0.000000,0.0,0.307692,0.384615,0.307692
2,2002-04-01,ATL,1.166667,0.133333,0.466667,0.400000,0.083333,0.166667,0.142857,0.0,0.300000,0.366667,0.333333
3,2002-04-01,PHI,0.333333,0.090909,0.227273,0.681818,0.133333,0.066667,0.000000,0.0,0.272727,0.363636,0.363636
4,2002-04-01,BOS,0.562500,0.166667,0.300000,0.533333,0.187500,0.250000,0.000000,0.0,0.225806,0.419355,0.354839
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98969,2022-10-05,SFG,1.875000,0.258065,0.483871,0.258065,0.000000,0.375000,0.000000,0.0,0.161290,0.580645,0.258065
98970,2022-10-05,LAA,1.333333,0.222222,0.444444,0.333333,0.111111,0.111111,0.000000,0.0,0.148148,0.629630,0.222222
98971,2022-10-05,OAK,0.900000,0.095238,0.428571,0.476190,0.100000,0.100000,0.000000,0.0,0.095238,0.714286,0.190476
98972,2022-10-05,DET,0.615385,0.160000,0.320000,0.520000,0.076923,0.000000,0.250000,0.0,0.440000,0.400000,0.160000


In [ ]:
games_f

,Game ID,Date,Season,Home,Team,Home Team,Away Team,Home Score,Away Score,Starting Pitcher,Outcome,Target Variable,Score_<=-5,Score_-3 to -4,Score_-1 to -2,Score_1 to 2,Score_3 to 4,Score_>=5
0,2002-04-01_STL_COL_10-2,2002-04-01,2002,False,COL,STL,COL,10,2,MHampton,8,>=5,0,0,0,0,0,1
1,2002-04-03_STL_COL_3-6,2002-04-03,2002,False,COL,STL,COL,3,6,DNeagle,-3,-3 to -4,0,1,0,0,0,0
2,2002-04-04_STL_COL_1-6,2002-04-04,2002,False,COL,STL,COL,1,6,JThomson,-5,<=-5,1,0,0,0,0,0
3,2002-04-05_LAD_COL_9-0,2002-04-05,2002,False,COL,LAD,COL,9,0,SChacon,9,>=5,0,0,0,0,0,1
4,2002-04-06_LAD_COL_9-2,2002-04-06,2002,False,COL,LAD,COL,9,2,JJennings,7,>=5,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98969,2022-10-01_MIL_MIA_3-4,2022-10-01,2022,False,MIA,MIL,MIA,3,4,ECabrera,-1,-1 to -2,0,0,1,0,0,0
98970,2022-10-02_MIL_MIA_3-4,2022-10-02,2022,False,MIA,MIL,MIA,3,4,PLopez,-1,-1 to -2,0,0,1,0,0,0
98971,2022-10-03_MIA_ATL_4-0,2022-10-03,2022,True,MIA,MIA,ATL,4,0,JLuzardo,4,3 to 4,0,0,0,0,1,0
98972,2022-10-04_MIA_ATL_1-2,2022-10-04,2022,True,MIA,MIA,ATL,1,2,BGarrett,-1,-1 to -2,0,0,1,0,0,0


In [ ]:
sp_adv_y_f

,Season,Team,Starting Pitcher,sp/yr K/9,sp/yr BB/9,sp/yr K/BB,sp/yr HR/9,sp/yr K%,sp/yr BB%,sp/yr K-BB%,sp/yr AVG,sp/yr WHIP,sp/yr BABIP,sp/yr LOB%,sp/yr FIP,sp/yr xFIP
0,2002,COL,MHampton,3.727612,4.583955,0.813187,1.208955,0.088305,0.108592,-0.020286,0.312757,1.785448,0.317757,0.653215,5.525507,5.236269
1,2002,NYY,RClemens,9.600000,3.150000,3.047619,0.900000,0.250000,0.082031,0.167969,0.250000,1.305556,0.315574,0.682657,3.295408,3.317903
2,2002,CHC,JLieber,5.553191,0.765957,7.250000,0.957447,0.149485,0.020619,0.128866,0.276673,1.170213,0.295503,0.703448,3.387606,3.604062
3,2002,SEA,JMoyer,5.735549,1.950867,2.940000,1.092486,0.157895,0.053706,0.104189,0.230233,1.075145,0.243902,0.771350,4.032884,4.289525
4,2002,DET,JWeaver,5.547945,2.441096,2.272727,0.295890,0.147348,0.064833,0.082515,0.242950,1.191781,0.277635,0.698779,3.167554,4.250490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6860,2022,TOR,DPhelps,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,1.000000,0.333333,1.000000,6.112431,6.112431
6861,2022,MIA,EHernandez,7.907143,2.507143,3.153846,2.700000,0.198068,0.062802,0.135266,0.265957,1.350000,0.264706,0.668103,6.283860,5.016802
6862,2022,SDP,CStammen,9.000000,9.000000,1.000000,3.000000,0.166667,0.166667,0.000000,0.400000,3.000000,0.454545,0.526316,8.445764,6.086257
6863,2022,ATL,JStephens,0.000000,6.000000,0.000000,0.000000,0.000000,0.133333,-0.133333,0.384615,2.333333,0.384615,0.571429,5.112431,6.592800


In [ ]:
# merge sp_adv_y_f with the games data on the 'Season', 'team', and 'starting pitcher' column
merged = pd.merge(games_f, sp_adv_y_f, on=['Season', 'Team', 'Starting Pitcher'])

In [ ]:
# merge sp_bat_y_f with the games data on the 'Season', 'team', and 'starting pitcher' column
merged = pd.merge(merged, sp_bat_y_f, on=['Season', 'Team', 'Starting Pitcher'])

In [ ]:
# merge rp_adv_y_f with the games data on the 'Season', 'team' column
merged = pd.merge(merged, rp_adv_y_f, on=['Season', 'Team'])

In [ ]:
# merge rp_bat_y_f with the games data on the 'Season', 'team' column
merged = pd.merge(merged, rp_bat_y_f, on=['Season', 'Team'])

In [ ]:
# merge team_adv_bat_y_f with the games data on the 'Season', 'team' column
merged = pd.merge(merged, team_adv_bat_y_f, on=['Season', 'Team'])

In [ ]:
# merge team_bat_bat_y_f with the games data on the 'Season', 'team' column
merged = pd.merge(merged, team_bat_bat_y_f, on=['Season', 'Team'])

In [ ]:
# merge team_def_data_f with the games data on the 'Season', 'team' column
merged = pd.merge(merged, team_def_data_f, on=['Season', 'Team'])

In [ ]:
# save merged as csv file
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/merged.csv')
merged.to_csv(filepath, index=False)

In [10]:
# load merged
filepath = Path('/content/drive/MyDrive/Colab Notebooks/Projects/NeuralNet/blogs/Multinomial Logistic Regression/data files/fangraphs data/merged.csv')
merged = pd.read_csv(filepath)

In [11]:
merged

,Game ID,Date,Season,Home,Team,Home Team,Away Team,Home Score,Away Score,Starting Pitcher,...,rGFP,rPM,DRS,ARM,DPR,RngR,ErrR,UZR,UZR/150,Def
0,2002-04-01_STL_COL_10-2,2002-04-01,2002,False,COL,STL,COL,10,2,MHampton,...,0,0,0,4.3,-1.3,5.8,-7.6,1.2,0.7,-6.8
1,2002-04-07_LAD_COL_6-4,2002-04-07,2002,False,COL,LAD,COL,6,4,MHampton,...,0,0,0,4.3,-1.3,5.8,-7.6,1.2,0.7,-6.8
2,2002-04-12_COL_ARI_3-8,2002-04-12,2002,True,COL,COL,ARI,3,8,MHampton,...,0,0,0,4.3,-1.3,5.8,-7.6,1.2,0.7,-6.8
3,2002-04-17_COL_LAD_3-6,2002-04-17,2002,True,COL,COL,LAD,3,6,MHampton,...,0,0,0,4.3,-1.3,5.8,-7.6,1.2,0.7,-6.8
4,2002-04-24_CIN_COL_4-3,2002-04-24,2002,False,COL,CIN,COL,4,3,MHampton,...,0,0,0,4.3,-1.3,5.8,-7.6,1.2,0.7,-6.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98473,2022-07-23_PIT_MIA_1-0,2022-07-23,2022,False,MIA,PIT,MIA,1,0,MMeyer,...,6,3,11,6.5,2.4,-8.6,2.4,2.6,2.0,6.4
98474,2022-07-30_MIA_NYM_0-4,2022-07-30,2022,True,MIA,MIA,NYM,0,4,NNeidert,...,6,3,11,6.5,2.4,-8.6,2.4,2.6,2.0,6.4
98475,2022-08-13_MIA_ATL_2-6,2022-08-13,2022,True,MIA,MIA,ATL,2,6,TNance,...,6,3,11,6.5,2.4,-8.6,2.4,2.6,2.0,6.4
98476,2022-08-26_MIA_LAD_6-10,2022-08-26,2022,True,MIA,MIA,LAD,6,10,TNance,...,6,3,11,6.5,2.4,-8.6,2.4,2.6,2.0,6.4


In [ ]:
for col in merged.columns:
    print(col)

Game ID
Date
Season
Home
Team
Home Team
Away Team
Home Score
Away Score
Starting Pitcher
Outcome
Target Variable
Score_<=-5
Score_-3 to -4
Score_-1 to -2
Score_1 to 2
Score_3 to 4
Score_>=5
sp/yr K/9
sp/yr BB/9
sp/yr K/BB
sp/yr HR/9
sp/yr K%
sp/yr BB%
sp/yr K-BB%
sp/yr AVG
sp/yr WHIP
sp/yr BABIP
sp/yr LOB%
sp/yr FIP
sp/yr xFIP
sp/yr GB/FB
sp/yr LD%
sp/yr GB%
sp/yr FB%
sp/yr IFFB%
sp/yr HR/FB
sp/yr IFH%
sp/yr BUH%
sp/yr Soft%
sp/yr Med%
sp/yr Hard%
rp/yr K/9
rp/yr BB/9
rp/yr K/BB
rp/yr HR/9
rp/yr K%
rp/yr BB%
rp/yr K-BB%
rp/yr AVG
rp/yr WHIP
rp/yr BABIP
rp/yr LOB%
rp/yr FIP
rp/yr xFIP
rp/yr GB/FB
rp/yr LD%
rp/yr GB%
rp/yr FB%
rp/yr IFFB%
rp/yr HR/FB
rp/yr IFH%
rp/yr BUH%
rp/yr Soft%
rp/yr Med%
rp/yr Hard%
/yr BB%
/yr K%
/yr BB/K
/yr AVG
/yr OBP
/yr SLG
/yr OPS
/yr ISO
/yr BABIP
/yr wRC
/yr wRAA
/yr wOBA
/yr wRC+
/yr GB/FB
/yr LD%
/yr GB%
/yr FB%
/yr IFFB%
/yr HR/FB
/yr IFH%
/yr BUH%
/yr Soft%
/yr Med%
/yr Hard%
Inn
rSZ
rCERA
rSB
rGDP
rARM
rGFP
rPM
DRS
ARM
DPR
RngR
ErrR
UZR
UZR/150
Def


In [ ]:
# write a function that can take in a dataframe of stats per game and spit out 
# a dataframe of average stats over last 'x' amount of days

# cols = []
def get_rolling_avg(df, days, names=False):

    # convert days param to string and add 'D' character to represent calendar days when passed into rolling function
    days = str(days)+'D'
    # make sure date column in of datetime type
    df['Date'] = pd.to_datetime(df['Date'])
    if names:
        rolling_avg = df.groupby(['Team', 'Starting Pitcher']).rolling('10D', on='Date', min_periods=1).mean()
    else:
        rolling_avg = df.groupby('Team').rolling('10D', on='Date', min_periods=1).mean()
    # add 'x' day avg to column name
    rolling_avg = rolling_avg.add_prefix(f'{days} day ')
    rolling_avg = rolling_avg.reset_index()
    if names:
        rolling_avg = rolling_avg.drop(columns='level_2')
    else:
        rolling_avg = rolling_avg.drop(columns='level_1')
    return rolling_avg

In [ ]:
# set days variable to calculate rolling average of
days = 10
sp_days = 15

In [ ]:
# get rolling averages for for all categories using per game databases
sp_adv_g_avg = get_rolling_avg(sp_adv_g_f, sp_days, names=True)
sp_bat_g_avg = get_rolling_avg(sp_bat_g_f, sp_days, names=True)
rp_adv_g_avg = get_rolling_avg(rp_adv_g_f, days)
rp_bat_g_avg = get_rolling_avg(rp_bat_g_f, days)
team_adv_bat_g_avg = get_rolling_avg(team_adv_bat_g_f, days)
team_bat_bat_g_avg = get_rolling_avg(team_bat_bat_g_f, days)

In [9]:
def Regressor(X, Y, iterations, learning_rate, use_GPU=False):
    
    if use_GPU:
        xp = cp
    else:
        xp = np

    # if use_GPU:
    #     X = cp.array(X)
    #     Y = cp.array(Y)
    #     Theta = cp.random.randn(X.shape[1], 1)
    # else:
    X = xp.array(X)
    Y = xp.array(Y)
    Theta = xp.random.randn(X.shape[1], 1)

    mses = []
    for i in range(iterations):
        # if use_GPU:
        #     Yh = X.dot(Theta)
        #     residual = Y - Yh
        #     mse = cp.sum((cp.square(residual))) / len(Y)
        #     mses.append(mse)
        #     Theta -= (learning_rate / len(Yh)) * (-2 * X.T.dot(residual))
        # else:
        Yh = X.dot(Theta)
        residual = Y - Yh
        mse = xp.sum((xp.square(residual))) / len(Y)
        mses.append(mse)
        Theta -= (learning_rate / len(Yh)) * (-2 * X.T.dot(residual))

    return mses, Theta

In [12]:
def r_squared(Y,predictions):
    # calculate residual sum of squares
    residuals = Y-predictions
    SSR = np.sum(residuals**2)
    # calculate total sum of squares
    SST=np.sum(np.square(Y-Y.mean()))
    # calculate coefficient of determination
    r_sq = 1 - (SSR / SST)
    
    return r_sq

In [13]:
def VIF(r_sq):
    return 1/(1-r_sq)

In [8]:
# set xp to numpy or cupy depending on if using GPU
class StandardScaler():
    def __init__(self, use_GPU=False):
        self.mean = None
        self.standard_deviation = None
        self.use_GPU = use_GPU

    def fit(self, data):
        if self.use_GPU:
            xp = cp
        else:
            xp = np
        # if self.use_GPU:
        #     data = cp.array(data)
        #     self.mean = cp.mean(data, axis=0)
        #     self.standard_deviation = cp.sqrt(cp.mean(cp.square(data - self.mean), axis=0))
        # else:
        self.mean = xp.mean(data, axis=0)
        self.standard_deviation = xp.sqrt(xp.mean(xp.square(data - self.mean), axis=0))

    def transform(self, data):
        # if self.use_GPU:
        #     xp = cp
        # else:
        #     xp = np
        # if self.use_GPU:
        #     data = cp.array(data)
        #     return (data - self.mean) / self.standard_deviation
        # else:
        return (data - self.mean) / self.standard_deviation


In [6]:
def collinearity_test(data, iterations, learning_rate, use_GPU=False):
    r2s = []
    VIFs = []

    # set xp to numpy or cupy depending on if using GPU
    if use_GPU:
        xp = cp
    else:
        xp = np
    
    for col in data.columns:
        X = data.drop(columns=col).to_numpy()
        Y = data[col].to_numpy().reshape((-1, 1))

        scaler = StandardScaler(use_GPU)
        scaler.fit(X)
        X = scaler.transform(X)

        # if use_GPU:
        #     X = cp.column_stack((cp.ones((X.shape[0], 1)), X))
        #     _, Theta = Regressor(X, Y, iterations, learning_rate, use_GPU)
        #     Theta = cp.asnumpy(Theta)
        #     Y = cp.asnumpy(Y)
        #     X = cp.asnumpy(X)
        # else:
        X = xp.column_stack((xp.ones((X.shape[0], 1)), X))
        _, Theta = Regressor(X, Y, iterations, learning_rate, use_GPU)

        # calculate r-squared
        r_sq = r_squared(Y,X.dot(Theta))
        r2s.append(r_sq)
        # calculate VIF
        vif = VIF(r_sq)
        VIFs.append(vif)

    return r2s, VIFs

In [14]:
# write a function that takes in a pandas dataframe, adds a column for the bias term, and returns an input and target dataframe
def format_input(df, collinearity_test=False):
    if collinearity_test:
        input = df.drop(columns=['Game ID','Date','Season','Home','Team','Home Team','Away Team','Home Score','Away Score','Starting Pitcher','Target Variable','Score_<=-5','Score_-3 to -4','Score_-1 to -2','Score_1 to 2','Score_3 to 4','Score_>=5'])
        return input
    else:
        df = df.drop(columns=['Game ID','Date','Season','Home','Team','Home Team','Away Team','Home Score','Away Score','Starting Pitcher','Outcome','Target Variable'])
        input = df.drop(columns=['Score_<=-5','Score_-3 to -4','Score_-1 to -2','Score_1 to 2','Score_3 to 4','Score_>=5'])
        target = df[['Score_<=-5','Score_-3 to -4','Score_-1 to -2','Score_1 to 2','Score_3 to 4','Score_>=5']]
        return input, target


In [15]:
# pass in dataframe of input data and get a numpy array with bias term added
input = format_input(merged, True)

In [16]:
input

,Outcome,sp/yr K/9,sp/yr BB/9,sp/yr K/BB,sp/yr HR/9,sp/yr K%,sp/yr BB%,sp/yr K-BB%,sp/yr AVG,sp/yr WHIP,...,rGFP,rPM,DRS,ARM,DPR,RngR,ErrR,UZR,UZR/150,Def
0,8,3.727612,4.583955,0.813187,1.208955,0.088305,0.108592,-0.020286,0.312757,1.785448,...,0,0,0,4.3,-1.3,5.8,-7.6,1.2,0.7,-6.8
1,2,3.727612,4.583955,0.813187,1.208955,0.088305,0.108592,-0.020286,0.312757,1.785448,...,0,0,0,4.3,-1.3,5.8,-7.6,1.2,0.7,-6.8
2,-5,3.727612,4.583955,0.813187,1.208955,0.088305,0.108592,-0.020286,0.312757,1.785448,...,0,0,0,4.3,-1.3,5.8,-7.6,1.2,0.7,-6.8
3,-3,3.727612,4.583955,0.813187,1.208955,0.088305,0.108592,-0.020286,0.312757,1.785448,...,0,0,0,4.3,-1.3,5.8,-7.6,1.2,0.7,-6.8
4,1,3.727612,4.583955,0.813187,1.208955,0.088305,0.108592,-0.020286,0.312757,1.785448,...,0,0,0,4.3,-1.3,5.8,-7.6,1.2,0.7,-6.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98473,1,9.000000,3.000000,3.000000,3.000000,0.230769,0.076923,0.153846,0.291667,1.500000,...,6,3,11,6.5,2.4,-8.6,2.4,2.6,2.0,6.4
98474,-4,5.400000,0.000000,3.000000,1.800000,0.150000,0.000000,0.150000,0.250000,1.000000,...,6,3,11,6.5,2.4,-8.6,2.4,2.6,2.0,6.4
98475,-4,12.600000,5.400000,2.333333,1.800000,0.318182,0.136364,0.181818,0.210526,1.400000,...,6,3,11,6.5,2.4,-8.6,2.4,2.6,2.0,6.4
98476,-4,12.600000,5.400000,2.333333,1.800000,0.318182,0.136364,0.181818,0.210526,1.400000,...,6,3,11,6.5,2.4,-8.6,2.4,2.6,2.0,6.4


In [ ]:
# run a collinearity test on input features
r2s, VIFs = collinearity_test(input, 1000, 0.01, use_GPU=True)

In [ ]:
VIFs

[]

In [ ]:
collin = pd.DataFrame()
collin['Feature Variable'] = input.columns
collin['r2s'] = r2s
collin['VIFs'] = VIFs

In [ ]:
collin

,Feature Variable,r2s,VIFs
0,Outcome,-0.007042,0.993007
1,sp/yr K/9,0.951017,20.415215
2,sp/yr BB/9,0.944914,18.153534
3,sp/yr K/BB,0.686067,3.185393
4,sp/yr HR/9,0.665106,2.986021
...,...,...,...
84,RngR,0.996423,279.536391
85,ErrR,0.873799,7.923854
86,UZR,0.997959,489.851610
87,UZR/150,0.982335,56.608039


In [ ]:
# remove RngR and UZR due to extremely high VIF's and rerun collinearity test
input = input.drop(columns=['RngR','UZR'])
r2s, VIFs = collinearity_test(input, 1000, 0.01)
collin = pd.DataFrame()
collin['Feature Variable'] = input.columns
collin['r2s'] = r2s
collin['VIFs'] = VIFs
collin

,Feature Variable,r2s,VIFs
0,Outcome,-0.003554,0.996459
1,sp/yr K/9,0.941246,17.020215
2,sp/yr BB/9,0.919670,12.448638
3,sp/yr K/BB,0.691294,3.239331
4,sp/yr HR/9,0.724792,3.633622
...,...,...,...
82,ARM,0.786947,4.693658
83,DPR,0.590772,2.443623
84,ErrR,0.358331,1.558435
85,UZR/150,0.887250,8.869177


In [21]:
## code k-fold cross validation ##
def cross_validate(X, Y, iterations, learning_rate, use_GPU=False):

    # set xp to numpy or cupy depending on if using GPU
    if use_GPU:
        xp = cp
    else:
        xp = np

    # transform pd database into xp (np/cp) arrays
    X = xp.array(X)
    Y = xp.array(Y)

    #scale data
    scaler = StandardScaler(use_GPU)
    scaler.fit(X)
    X = scaler.transform(X)

    # add column of ones to X
    X = xp.column_stack((xp.ones((X.shape[0],1)),X))

    # initialize theta term
    theta = xp.random.randn(X.shape[1],1)

    # split data into train and validation set
    ## shuffle data
    p = xp.random.permutation(len(X))
    X = X[p]
    Y = Y[p]

    ## split
    test_percent = 10
    test_size = len(X) // test_percent

    x_train = X[:-test_size]
    y_train = Y[:-test_size]
    x_test = X[-test_size:]
    y_test = Y[-test_size:]

    # split train into k folds
    folds = 10
    fold_size = len(x_train) // folds

    # initialize array to store mses
    mses = []

    for i in range(iterations):
        # reset fold mses array to be empty
        fold_mses = []

        for k in range(folds):
            # get range of indices to use as test fold
            test_fold = xp.arange(k*fold_size,(k+1)*fold_size)
            # set train and test data according to current test fold
            x_train_fold = xp.delete(x_train, test_fold, axis=0)
            y_train_fold = xp.delete(y_train, test_fold, axis=0)
            x_test_fold = xp.take(x_train, test_fold, axis=0)
            y_test_fold = xp.take(y_train, test_fold, axis=0)

            # run regression on training folds
            Yh = x_train_fold.dot(theta)
            residual = y_train_fold - Yh
            train_mse = xp.sum((xp.square(residual)))/len(y_train_fold)
            theta -= (learning_rate/len(Yh))*(-2*x_train_fold.T.dot(residual))

            # test on test fold
            Yh = x_test_fold.dot(theta)
            residual = y_test_fold - Yh
            test_mse = xp.sum((xp.square(residual)))/len(y_test_fold)

            fold_mses.append([train_mse, test_mse])

        mses.append(xp.mean(fold_mses, axis=0))
        # if i % 10 == 0:
        #     print(f'Iteration {i}: {mses[-1]}')

    # test validation data to determine true MSE
    Yh = x_test.dot(theta)
    residual = y_test - Yh
    mse = np.sum((xp.square(residual)))/len(y_test)
    # print(f'Final MSE on validation data: {mse}')

    return mse, mses, {'x_train':x_train, 'y_train':y_train, 'x_test':x_test, 'y_test':y_test, 'theta':theta}

In [22]:
# The r-squared of the outcome variable is lower....and VIFs are still all over the place.  I am going to
# try a simple regression model to see how well I can predict the outcome variable to get a baseline

mse, mses, data_dict = cross_validate(X=input.drop(columns='Outcome'), Y=input['Outcome'], iterations=1000, learning_rate=0.01, use_GPU=True)

AttributeError: ignored